# TD5 - Chia nhóm các bài báo thời sự - xã hội trên VNExpress

## Mô tả

Ở TD3, chúng ta đã học kĩ năng thu thập dữ liệu từ Internet và xử lí dữ liệu để thu được các đoạn văn bản tương đối "sạch". Trong TD này, ta sẽ tiến hành chia nhóm các văn bản này.

### Tải dữ liệu

Khi chạy chương trình ở TD3 cho năm 2017 với thể loại **Thời sự**, ta thu được dữ liệu dưới dạng thư mục `RawData_News` <a href="https://drive.google.com/drive/folders/1u2IQzbDPvZ7Q_bE_uHMGkt5n9YF5QnmG?usp=sharing">ở đây</a>. Thư mục này gồm 12 file ứng với các tháng. Bạn cần tải thư mục này và lưu tại vị trí `Lesson5/TD/RawData`. Ta cũng cần làm việc với một danh sách các từ trong tiếng Việt, được trích xuất từ từ điển tiếng Việt của Hoàng Phê ([1]), nằm ở thư mục `VietnameseDictionary`. Bạn cần tải thư mục này và lưu tại `Lesson5/TD/VietnameseDictionary`. Cuối cùng, bạn có thể tải thư mục `FullData`, trong đó chứa kết quả bước 1 của TD trong trường hợp bạn không thực hiện thành công bước này.

### Quan sát sơ lược dữ liệu

Ta nhắc lại rằng ở TD3, mỗi bài báo của VNExpress được trích lọc tiêu đề, đoạn giới thiệu và nội dung. Ba thành phần này của mỗi bài báo đã được chúng ta lưu bằng một hàng của file dữ liệu, chúng cách nhau bởi hai khoảng trắng tab (`"\t\t"`). Riêng với thành phần thứ ba (nội dung của bài báo), các đoạn văn cách nhau bởi một khoảng trắng tab (`"\t"`).

Bạn có thể xem ví dụ từ <a href="https://drive.google.com/drive/folders/1nmsx4QAUwy2gQKMxqoTqshmTESoHUmYD">RawData_News/News_012017.txt</a>.

### Mục tiêu

Từ 12 file dữ liệu "thô" này, ta sẽ thực hiện việc tìm hiểu các bài báo thể thao năm 2017 của vnexpress có thể được chia thành những nhóm nào dựa trên sự tương đồng về nội dung. Ta sẽ thực hiện theo quy trình sau:

- Bước 1: Preprocessing 1 - Biến bài báo thành bag-of-words
 - Ghép 12 file dữ liệu thành file duy nhất và xoá các hàng trắng. File này tương tự như `FullData/News2017_Solution.txt`.
 - Tách nội dung mỗi bài báo thành một túi từ (bag of words), gồm mỗi từ và các tần số của nó trong mỗi bài báo, và lưu nó vào một file. File này tương tự như `FullData/FrequencyByNewsArticle_Solution.txt`
 - Tính tần số tổng cộng của các từ trên tất cả các bài báo và lưu nó vào một file. File này tương tự như `GlobalNewsFrequency_Solution.txt`. File này sẽ giúp ta xác định những từ nào là thông dụng, ít thông dụng trong tiếng Việt để xử lí ở các phần sau.
 
- Bước 2: Preprocessing 2 - Chọn biến quan trọng và biến bag-of-words thành vectors
 - Từ kết quả bước 1, ta sẽ chọn ra những từ được xem là quan trọng để phân loại các bài báo thể thao, sau đó trên cở sở này biến mỗi bài báo thành một vector trong $[0, 1]^d$ ($d \in \mathbf N$ là số từ quan trọng được lựa chọn, cũng chính là số chiều của không gian). 
 - Do mỗi vector có số lượng 0 rất lớn, ta có thể lưu tất cả các vector dưới dạng một ma trận sparse thông qua scipy.sparse.

- Bước 3: Chia nhóm với K-Means. 

 Đến bây giờ khi đã có các vector, ta sẽ dùng K-Means để chia nhóm, rồi thử thay đổi các tham số, rồi phân tích kết quả.

- Bước 4: Thử thực hiện một số phương pháp khác để so sánh kết quả
 - Cho $k$ thay đổi (3, 4, 6, 8, 10, 20, ...)
 - Thử với khoảng cách Euclide và cosine
 - Chia nhóm với Hierarchical Clustering.
 - Bạn cũng có thể tự do thử với các mô hình khác có sẵn trong Python và so sánh hiệu quả của các phương pháp.
 
Bạn cần hoàn thành các hàm trong file <a href="https://raw.githubusercontent.com/riduan91/DSC101/master/Lesson5/TD/SportArticlesClustering.py">SportArticlesClustering.py</a>  ứng với yêu cầu của các bài tập, và test theo chỉ dẫn ở từng bài tập.

Bạn có thể tham khảo lời giải ở <a  href="https://raw.githubusercontent.com/riduan91/DSC101/master/Lesson5/TD/SportArticlesClustering_Solution.py">SportArticlesClustering_Solution.py</a>  sau khi hoàn thành.

In [1]:
from SportArticlesClustering_Solution import *
import pandas as pd

Dưới đây là các hằng số cơ bản chỉ đường dẫn đến các tập tin sẽ làm việc. Khi test, nhớ copy và chạy các dòng này.

In [2]:
RAW_DATA_FOLDER = "RawData/"
ALL_DATA_FILE = "FullData/News2017.txt"
WORDLIST_FILE = "VietnameseDictionary/WordList.txt"
FREQUENCY_FILE = "FullData/FrequencyByNewsArticle.txt"
GLOBAL_FREQUENCY_FILE = "FullData/GlobalNewsFrequency.txt"

## Phần 1. Preprocessing 1 - Chuyển đổi bài báo thành túi từ

### Bài 1. Tổng hợp các bài báo theo tháng thành file chung

Giả sử trong thư mục **`raw_data_folder`** chứa các file dữ liệu con (trong trường hợp của chúng ta: 12 file, mỗi file ứng với một tháng), và các file này có dạng như output ở TD3. Một số hàng trong file có thể trắng do bài báo không có nội dung. 

*Viết hàm **`concatenateDataFiles(raw_data_folder, full_data_folder, all_data_file)`** nhận đối số là tên thư mục **`raw_data_folder`**, nối tất cả dữ liệu trong các file này thành một file duy nhất tại đường dẫn **`all_data_file`**, đồng thời xoá tất cả các hàng trắng trong file. *

Bạn nên nối các file theo thứ tự tên của chúng, để các bài báo xuất hiện theo thứ tự từ tháng 1 đến tháng 12.

Đoạn code dưới đây giúp test hàm của bạn.

In [3]:
concatenateDataFiles(RAW_DATA_FOLDER, ALL_DATA_FILE)
data = pd.read_csv(ALL_DATA_FILE, sep="\t\t", header = None)
len(data)

6748

In [4]:
data.head()

,0,1,2
0,Cơ quan khí tượng triều Nguyễn,Ngoài việc xác định khí tượng thời tiết có gì ...,"Sau khi lên ngôi năm 1802, vua Gia Long cho th..."
1,Những chính sách nổi bật có hiệu lực từ tháng 2,Người chở hàng hóa không che chắn bị phạt đến ...,Chế độ bồi dưỡng cho cán bộ tiếp công dân\tThe...
2,Đường vành đai Hà Nội ùn tắc ngày mùng 4 Tết,"Chưa hết kỳ nghỉ Tết, cửa ngõ phía Nam Hà Nội ...","Chiều mùng 4 tết Đinh Dậu (31/1), đường dẫn và..."
3,Ông Đinh La Thăng: 'CSGT TP HCM phải lấy lại h...,"Theo Bí thư Thành ủy TP HCM, một số cán bộ chi...",Thăm và làm việc với Phòng Cảnh sát giao thông...
4,Người dân dựng biển cảnh báo 'Công an bắn tốc độ',"Tấm bảng được cột vào thùng rác, dựng thêm một...",Mạng xã hội hôm 30/1 đăng tải tấm biển bằng ca...


In [5]:
data.tail()

,0,1,2
6743,Ông Nguyễn Thiện Nhân: 'Thành ủy TP HCM sẽ xem...,"Theo Bí thư Thành ủy TP HCM, năm nay số cán bộ...","Ngày 30/11, phát biểu khai mạc Hội nghị lần th..."
6744,Không chỉ định thầu xây sân bay Long Thành,"""Chúng ta cố gắng đấu thầu dự án để đảm bảo tố...",Tại cuộc họp bàn công tác triển khai dự án sân...
6745,Người lái xe container bằng chân tại Sài Gòn b...,"Tài xế phân bua ""chân bị tê nên gác lên vô lăn...","Ngày 1/12, Phòng CSGT TP HCM cho biết đã làm v..."
6746,"Hai người tử vong, xe máy nát vụn sau cú đâm đ...","Sau cú đâm đối đầu, cả hai xe máy bị nát vụn, ...","Hơn 19h tối 30/11 tại quốc lộ 6, (đoạn qua địa..."
6747,Bốn thứ trưởng nghỉ hưu từ ngày 1/12,Thủ tướng Nguyễn Xuân Phúc ký quyết định nghỉ ...,Ông Thạch Dư được bổ nhiệm làm Thứ trưởng Bộ N...


### Bài 2. Hàm phụ [1]: Tách đoạn văn thành câu

Ta bắt đầu thực hiện việc chính của preprocessing 1: biến nội dung mỗi bài báo thành túi từ.

Trong TD3, ta đã chuyển đổi nội dung bài báo thành các "từ" với giả sử rằng mỗi từ luôn là một từ đơn. Ví dụ "long lanh" được xem là hai từ, "long" và "lanh". Điều này không thực tế đối với tiếng Việt. Do vậy ta cần thực hiện một quy trình "gần đúng" như sau:

- Biến mỗi đoạn văn thành câu
- Biến mỗi câu thành từng "thành phần"
- Biến mỗi thành phần thành từng tiếng (tương đương từ đơn)
- Ghép các tiếng thành từ (từ đơn, từ ghép hoặc từ láy)

Lưu ý rằng quy trình này chỉ là gần đúng, không nên hy vọng nó thực hiện chính xác 100% mong muốn của chúng ta.

Bài tập 2, 3, 4, 6 thực hiện 4 thao tác trên.

*Hãy viết hàm **`paragraphToSentences(paragraph)`** nhận đối số **`paragraph`** là một đoạn văn dưới dạng `str`, và biến nó thành một list các câu theo thuật toán sau: *

*- Xoá tất cả các kí tự '\xc2\xa0' có trong đoạn văn (đây là một kí tự vô dụng thường gặp ở vnexpress)*

*- Những nơi có dấu chấm theo sau bởi dấu cách (. ), ba chấm hoặc nhiều chấm theo sau bởi dấu cách (... ), chấm hỏi theo sau bởi dấu cách (? ), chấm than theo sau bởi dấu cách (! ) được xem là các vị trí kết thúc câu. Split đoạn văn thành các câu tại các vị trí đó.*

Đoạn code dưới đây giúp test hàm của bạn.

Đoạn văn gốc:
*Ngày 8/3, Văn phòng Trung ương Đảng có công văn về việc xử lý vụ Tổng công ty viễn thông Mobifone mua 95% cổ phần của Công ty cổ phần nghe nhìn Toàn Cầu (AVG). Theo đó, vừa qua, Ban Bí thư đã họp dưới sự chủ trì của Tổng Bí thư Nguyễn Phú Trọng để nghe Ban cán sự đảng Thanh tra Chính phủ báo cáo kết quả việc thanh tra dự án nêu trên! Ban Bí thư cho rằng, đây là một vụ việc rất nghiêm trọng, phức tạp, nhạy cảm, dư luận xã hội đặc biệt quan tâm. Thanh tra Chính phủ đã có nhiều cố gắng tiến hành thanh tra toàn diện, kết luận và báo cáo với Ban Bí thư... Ban Bí thư đề nghị Thường trực Chính phủ, Thanh tra Chính phủ chỉ đạo và chịu trách nhiệm về Kết luận thanh tra, sớm công bố Kết luận thanh tra theo quy định của pháp luật. Các cơ quan có trách nhiệm khẩn trương xem xét, xử lý vụ việc bảo đảm khách quan, chính xác theo quy định của Đảng và pháp luật Nhà nước với tinh thần kiên quyết, chặt chẽ, làm rõ đến đâu xử lý đến đó, đúng người, đúng vi phạm, đúng pháp luật và thu hồi tài sản Nhà nước bị thất thoát.*

In [6]:
p = "Ngày 8/3, Văn phòng Trung ương Đảng có công văn về việc xử lý vụ Tổng công ty viễn thông Mobifone mua " +\
    "95% cổ phần của Công ty cổ phần nghe nhìn Toàn Cầu (AVG). " +\
    "Theo đó, vừa qua, Ban Bí thư đã họp dưới sự chủ trì của Tổng Bí thư Nguyễn Phú Trọng để nghe Ban cán sự đảng Thanh tra " +\
    "Chính phủ báo cáo kết quả việc thanh tra dự án nêu trên!" +\
    "Ban Bí thư cho rằng, đây là một vụ việc rất nghiêm trọng, phức tạp, " +\
    "nhạy cảm, dư luận xã hội đặc biệt quan tâm. Thanh tra "+\
    "Chính phủ đã có nhiều cố gắng tiến hành thanh tra toàn diện, kết luận và báo cáo với Ban Bí thư... " +\
    "Ban Bí thư đề nghị Thường trực Chính phủ, Thanh tra Chính phủ chỉ đạo và chịu trách nhiệm về Kết luận thanh tra, " +\
    "sớm công bố Kết luận thanh tra theo quy định của pháp luật. Các cơ quan có trách nhiệm khẩn trương xem xét, " +\
    "xử lý vụ việc bảo đảm khách quan, chính xác theo quy định của Đảng và pháp luật Nhà nước với tinh thần kiên quyết, " +\
    "chặt chẽ, làm rõ đến đâu xử lý đến đó, đúng người, đúng vi phạm, đúng pháp luật và thu hồi tài sản Nhà nước bị thất thoát."
s = paragraphToSentences(p) # Kết quả là 1 list
pd.DataFrame(s) # Chuyển list thành DataFrame để dễ quan sát

,0
0,"Ngày 8/3, Văn phòng Trung ương Đảng có công vă..."
1,"Theo đó, vừa qua, Ban Bí thư đã họp dưới sự ch..."
2,Thanh tra Chính phủ đã có nhiều cố gắng tiến h...
3,"Ban Bí thư đề nghị Thường trực Chính phủ, Than..."
4,Các cơ quan có trách nhiệm khẩn trương xem xét...


### Bài 3. Hàm phụ [2]: Tách câu thành thành phần

Trong mỗi câu, ta có thể gặp các dấu câu khác như dấu phẩy, chấm phẩy, hai chấm, gạch ngang... Chúng chia câu thành các thành phần nhỏ hơn. Ví dụ, câu *"Theo đó, vừa qua, Ban Bí thư đã họp dưới sự chủ trì của Tổng Bí thư Nguyễn Phú Trọng để nghe Ban cán sự đảng Thanh tra Chính phủ báo cáo kết quả việc thanh tra dự án nêu trên!"* có thể được xem là tổng hợp của 3 thành phần *"Theo đó,", "vừa qua,", "Ban Bí thư đã họp dưới sự chủ trì của Tổng Bí thư Nguyễn Phú Trọng để nghe Ban cán sự đảng Thanh tra Chính phủ báo cáo kết quả việc thanh tra dự án nêu trên!"*

*Hãy viết hàm **`sentenceToSegments(sentence)`** nhận đối số **`sentence`** là một đoạn câu dưới dạng `str`, tức một phần tử của list trong output của bài 2, và biến nó thành một list các thành phần theo thuật toán tương tự như ở bài 2, nhưng đối với các dấu phẩy theo sau bởi dấu cách (, ), hai chấm theo sau bởi dấu cách (: ), chấm phẩy theo sau bởi dấu cách (; ), gạch ngang theo sau bởi dấu cách (- ), đóng ngoặc đơn theo sau bởi dấu cách () ), dấu cách theo sau bởi mở ngoặc đơn (( ).*

Đoạn code dưới đây giúp test hàm của bạn.

In [7]:
s = "Các cơ quan có trách nhiệm khẩn trương xem xét, xử lý vụ việc bảo đảm khách quan, chính xác; " +\
    "theo quy định của Đảng và pháp luật Nhà nước: với tinh thần kiên quyết, chặt chẽ, làm rõ đến đâu xử lý đến đó, " +\
    "đúng người, đúng vi phạm, đúng pháp luật và thu hồi tài sản Nhà nước bị thất thoát."
sg = sentenceToSegments(s) #Kết quả là  1 list
pd.DataFrame(sg)

,0
0,"Các cơ quan có trách nhiệm khẩn trương xem xét,"
1,"xử lý vụ việc bảo đảm khách quan,"
2,chính xác;
3,theo quy định của Đảng và pháp luật Nhà nước:
4,"với tinh thần kiên quyết,"
5,"chặt chẽ,"
6,"làm rõ đến đâu xử lý đến đó,"
7,"đúng người,"
8,"đúng vi phạm,"
9,đúng pháp luật và thu hồi tài sản Nhà nước bị ...


### Bài 4. Hàm phụ [3]: Tách thành phần thành tiếng
*Hãy viết hàm **`segmentToUnits(segment)`** nhận đối số **`segment`** là một thành phần dưới dạng `str`, tức một phần tử của list trong output của bài 3, và trả lại kết quả là một list tất cả các tiếng (từ đơn) sau khi bỏ hết các dấu câu, số và kí tự lạ.*

Regular expression (biểu thức chính quy) sau có thể giúp bạn: `r'[,;$\:\^\=\+\-\"\'\(\)\/\@\*\&\%\“\.{1,}\?\!\d]'` (bạn có thể thêm những dấu khác có thể gặp)

Đoạn code dưới đây giúp test hàm của bạn.

In [8]:
e = 'Cho phương trình "x^3 + 3mx^2 + 2x - 1 = 0" tìm m để phương trình có 3 nghiệm phân biệt.'
u = segmentToUnits(e) # Kết quả là 1 list
pd.DataFrame(u)

,0
0,Cho
1,phương
2,trình
3,x
4,mx
5,x
6,tìm
7,m
8,để
9,phương


### Bài 5. Tải (load) danh sách từ tiếng Việt

Việc tách câu thành đoạn, đoạn thành thành phần, thành phần thành tiếng có thể xem là các việc thuần tuý kĩ thuật. Thao tác còn lại: ghép tiếng thành từ để thực hiện được cần có sự trợ giúp của một từ điển được xem là chứa tất cả các từ (đơn, ghép, láy) trong tiếng Việt. Ta load từ điển trong bài này.


*Hãy viết hàm **`loadWordSet(wordlist_filename)`** nhận đối số **`wordlist_filename`** là đường dẫn của file chứa danh sách các từ (file này có dạng như `VietnameseDictionary/WordList.txt`), sau đó trả lại một **set** là tập hợp tất cả các từ trong file đó.*

Đoạn code dưới đây giúp test hàm của bạn.

In [9]:
s = loadWordSet(WORDLIST_FILE)
"nhân dân" in s, "người" in s, "công nghệ" in s

(True, True, True)

In [10]:
len(s)

29568

### Bài 6. Hàm phụ [4]: Ghép tiếng thành từ

Thông thường, không có từ điển nào liệt kê đủ mọi từ, bởi các từ mới luôn xuất hiện theo thời gian, và hàng loạt thuật ngữ, từ mượn, thành ngữ, danh từ riêng ... không phải lúc nào cũng xuất hiện trong từ điển.

Kể cả khi tồn tại một từ điển liệt kê đầy đủ các từ, thì việc ghép các từ đơn thành từ phức (ghép, láy) cũng không thể giải quyết tốt 100%, do hiện tượng đồng âm, kiêm nhiệm chức năng... của từ. Ví dụ, xét 2 câu:

- *"Mây được hình thành từ hơi nước."*

- *"Ta chia hình thành 3 phần bằng nhau."*

Trong câu 1, tổ hợp "hình thành" là một từ ghép, còn trong câu 2, tổ hợp "hình thành" là 2 từ đơn. Để biết câu đang xét thuộc trường hợp nào, đó là một bài toán data science khác vượt ngoài phạm vi TD này.

Vì vậy, ta chỉ có thể giải quyết gần đúng bài toán ghép tiếng thành từ. Chẳng hạn, với list các tiếng `['Ta', 'chia', 'hình', 'thành', 'phần', 'bằng', 'nhau']`, ta có thể chấp nhận để chương trình trả lại list các từ sau khi ghép `['ta', 'chia', 'hình thành', 'phần', 'bằng', 'nhau']`.

Một trong những thuật toán có thể ghép gần đúng tiếng thành từ là thuật toán **tham lam** hay **so khớp từ dài nhất** (longest matching) sau: 

- Đọc list các tiếng từ trái sang phải.

- Nếu một tiếng được viết hoa, **có mặt trong từ điển** nhưng không nằm kề một tiếng viết hoa nào, thì chuyển nó thành dạng viết thường. Các trường hợp khác giữ nguyên trạng thái viết hoa hoặc thường của nó: `['Trung', 'tâm', 'Hữu', 'nghị', 'Việt', 'Đức'] -> ['trung', 'tâm', 'hữu', 'nghị', 'Việt', 'Đức']`; `['Anh', 'và', 'Tây', 'Ban', 'Nha', 'và', 'Ireland', 'đều', 'thuộc', 'châu', 'Âu'] -> ['anh', 'và', 'Tây', 'Ban', 'Nha', 'và', 'Ireland', 'đều', 'thuộc', 'châu', 'âu']`.

- Ứng với mỗi tiếng, ghép nó với số lượng lớn nhất các tiếng ở sát bên phải nó sao cho tổ hợp tạo thành hoặc là một từ có mặt trong từ điển, hoặc là một dãy viết hoa liên tiếp cực đại. `['nhân', 'dân', 'ta', 'có', 'một', 'lòng', 'nồng', 'nàn', 'yêu', 'nước'] -> ['nhân dân', 'ta', 'có', 'một lòng', 'nồng nàn', 'yêu', 'nước']`; `['anh', 'và', 'Tây', 'Ban', 'Nha', 'và', 'Ireland', 'đều', 'thuộc', 'châu', 'âu'] -> ['anh', 'và', 'Tây Ban Nha', 'và', 'Ireland', 'đều', 'thuộc', 'châu', 'âu']`

 (tại các vị trí **nhân**, **một**, **nồng**; tổ hợp **nhân dân**, **một lòng**, **nồng nàn** có mặt trong từ điển còn **nhân dân ta**, **một lòng nồng**, **nồng nàn yêu** không có mặt trong từ điển, cũng không được viết hoa; tổ hợp "Tây Ban Nha" được viết hoa còn **Tây Ban Nha và** không được viết hoa, v.v.).

- Cuối cùng, nếu bản thân tiếng đó không có mặt trong từ điển và cũng không ghép được với tổ hợp nào ở sát bên phải để tạo từ, ta cũng xem nó là một từ: `['tích', 'xy', 'của', 'x', 'và', 'y'] -> ['tích', 'xy', 'của', 'x', 'và', 'y']` (**xy** trở thành từ dù nó không có trong từ điển)


*Hãy viết hàm **`unitsToWords(units, word_set)`** nhận đối số **`units`** là list các tiếng như output của bài 4 và **`word_set`** là danh sách các từ dưới dạng set như output của bài 5, sau đó thực hiện thuật toán tham lam nêu trên để trả lại list các từ được tạo thành.*

Đoạn code dưới đây giúp test hàm của bạn.

In [11]:
e = 'Cho phương trình "x^3 + 3mx^2 + 2x - 1 = 0" tìm m để phương trình có 3 nghiệm phân biệt.'
u = segmentToUnits(e)
s = loadWordSet(WORDLIST_FILE) # Kết quả là 1 set
w = unitsToWords(u, s) # Kết quả là 1 list
pd.DataFrame(w)

,0
0,Cho
1,phương trình
2,x
3,mx
4,x
5,tìm
6,m
7,để
8,phương trình
9,có


### Bài 7. Tổng hợp các bước: biến văn bản thành túi từ

Bây giờ ta có thể tổng hợp tất cả các bước từ các bài trên để từ file dữ liệu tổng hợp (`FullData/Sport2017_Solution.txt`) tạo ra các túi từ.

*Viết hàm **`getWordFrequencyFromArticles(data_file, word_list_file)`** nhận đối số **`data_file`** là đường dẫn của file dữ liệu tổng hợp và **`word_list_file`** là đường dẫn của file danh sách từ (`VietnameseDictionary/WordList.txt`), và trả lại một **list** có số phần tử bằng số bài báo trong **`data_file`**, mỗi phần tử là một **dict** của Python có dạng **từ : tần số xuất hiện của từ trong nội dung bài báo**).*

Đoạn code dưới đây giúp test hàm của bạn. (Thời gian chạy khoảng 1-2 phút)

In [12]:
freq_dict = getWordFrequencyFromArticles(ALL_DATA_FILE, WORDLIST_FILE) #Kết quả freq_dict là một list các dict
len(freq_dict)

6748

In [13]:
c1 = pd.Series(freq_dict[0].keys()) #freq_dict[0] là một dict, lấy tất cả các key (từ) của nó
c2 = pd.Series(freq_dict[0].values()) # Lấy tần số tương ứng
pd.DataFrame([c1, c2]) # Quan sát bằng DataFrame

,0,1,2,3,4,5,6,7,8,9,...,276,277,278,279,280,281,282,283,284,285
0,Bảo,và,lập,thời,đổ nát,tham mưu,xem,bóng,đầu,còn,...,quan,hàng,là,hoàn thành,ngày,kiêm,cấp,ghi,đuôi,Bắc Kỳ
1,1,5,1,5,1,1,5,1,2,3,...,14,2,6,1,2,1,2,1,1,1


Ví dụ trên cho thấy nội dung bài báo thứ nhất đã được chuyển đổi thành các bộ từ : tần số. Từ *"Bảo"* xuất hiện 1 lần, từ *"và"* xuất hiện 5 lần, từ *"tham mưu"* xuất hiện 1 lần v.v...

### Bài 8. Lưu túi từ vào file

Để các bước tiếp theo không phải thực hiện lại quá trình preprocessing1 này, ta lưu kết quả đã thực hiện vào file (Trong thực tế, người ta lưu vào cơ sở dữ liệu thay vì file). Ta sẽ cần 2 file:

- Một file như `FullData/FrequencyBySportArticle_Solution.txt` chứa tiêu đề các bài báo và túi từ tương ứng của nó.
- Một file như `FullData/GlobalSportFrequency_Solution.txt` chứa tổng tần số của tất cả các từ xuất hiện ít nhất trong 1 bài báo, lấy trên tất cả các bài báo.

*Viết hàm **`saveWordFrequencyToFile(data_file, word_list_file, frequency_file, global_frequency_file)`** nhận đối số **`data_file`**, **`word_list_file`** là file các bài báo và file danh sách từ, rồi thực hiện việc biến các bài báo thành list các túi từ rồi ghi vào **`frequency_file`** theo quy tắc: Mỗi hàng tương ứng là một bài báo gồm: tiêu đề bài báo, theo sau bởi hai dấu tab (`\t\t`), theo sau bởi các cặp từ:tần số (ở giữa là dấu hai chấm), các cặp này cách nhau bởi 1 dấu tab (`\t`). Ví dụ hàng đầu tiên có dạng: **tieude1**\t\t**tu1**:**tanso1**\t**tu2**:**tanso2**...\t**tuN**:**tansoN**\n. *

*Tiếp theo, hàm tính tổng tần số của các từ trên tất cả các bài báo và lưu vào **`global_frequency_file`** sao cho mỗi từ ứng với một hàng, mỗi hàng gồm từ, theo sau bởi dấu hai chấm, theo sau bởi tổng tần số tương ứng. Để thuận tiện cho tính toán sau này, bạn nên lưu theo thứ tự tần số giảm dần, hàng tương ứng với từ có tần số cao hơn xuất hiện trước.*

Đoạn code dưới đây giúp test hàm của bạn.

In [14]:
saveWordFrequencyToFile(ALL_DATA_FILE, WORDLIST_FILE, FREQUENCY_FILE, GLOBAL_FREQUENCY_FILE)

In [15]:
pd.read_csv(FREQUENCY_FILE, sep="\t\t", names=["Tiêu đề", "Túi từ"]).head()

,Tiêu đề,Túi từ
0,Cơ quan khí tượng triều Nguyễn,Bảo:1\tvà:5\tlập:1\tthời:5\tđổ nát:1\ttham mưu...
1,Những chính sách nổi bật có hiệu lực từ tháng 2,sai sót:1\ttham dự:2\tđang:1\tphụ cấp:2\tnhận:...
2,Đường vành đai Hà Nội ùn tắc ngày mùng 4 Tết,đón:1\tchôn chân:1\tChiều:1\tHưng:1\tKhuất Duy...
3,Ông Đinh La Thăng: 'CSGT TP HCM phải lấy lại h...,sức lực:1\tđông:3\tTP HCM:9\ttrở lại:1\tlần:1\...
4,Người dân dựng biển cảnh báo 'Công an bắn tốc độ',đông:1\tđang:1\tCông:1\ttrăm:2\tở:1\tchốt:2\td...


In [16]:
pd.read_csv(GLOBAL_FREQUENCY_FILE, sep=":", names=["Từ", "Tần số"]).head(20)

,Từ,Tần số
0,và,5973
1,được,5572
2,cho,5353
3,có,5264
4,trong,5128
5,đã,5107
6,của,5088
7,các,4840
8,không,4766
9,người,4706


## Phần 2: Preprocessing 2 - Số hoá túi từ

Từ thời điểm này, ta chỉ cần làm việc với 3 file, trong đó 2 file sau chính là kết quả của bước 1.
```python
WORDLIST_FILE = "VietnameseDictionary/WordList.txt"
FREQUENCY_FILE = "FullData/FrequencyBySportArticle.txt"
GLOBAL_FREQUENCY_FILE = "FullData/GlobalSportFrequency.txt"
```

Để thuận tiện cho việc thống nhất số liệu để so sánh kết quả với hướng dẫn của TD, bạn có thể dùng 2 file `..._Solution.txt` và đổi tên thành các file trên.


### Bài 9. Giảm số chiều bằng cách chọn các từ quan trọng

Để các thuật toán clustering có thể chạy được, ta phải biến các túi từ thành các vector, trong đó mỗi toạ độ thể hiện một từ. Vấn đề đầu tiên đặt ra: ta sử dụng hết tất cả các từ hiện có hay có thể chọn lọc trước một số từ quan trọng. Đây là bài toán giảm số chiều (reduction of dimension) trong machine learning. Ta chưa đi sâu vào các kĩ thuật cho bài toán này, mà sử dụng phương pháp đơn giản dựa vào quan sát file `FullData/GlobalSportFrequency.txt`.

Ta thấy:

- Những từ có tần số quá lớn thường không có tính phân loại, tức không quan trọng. Ví dụ các từ "và", "của", "hoặc"... có thể xuất hiện trong rất nhiều bài báo, không thể căn cứ vào đó để chia nhóm nội dung.

- Những từ có tần số quá nhỏ cũng không quan trọng. Đó là những từ chỉ xuất hiện đơn lẻ trong một vài bài báo, không đặc trưng cho một nhóm.

- Những từ xuất hiện đều đặn trong các bài báo (có tần số gần như nhau trong hầu hết tất cả bài báo) thường cũng không thể được dùng để phân loại các bài báo. 

Do đó, ta sẽ chọn ra những từ thoả mãn điều kiện sau:

- Có tần số nhỏ hơn hoặc bằng một giá trị chặn trên **upperbound**

- Có tần số lớn hơn hoặc bằng một giá trị chặn dưới **lowerbound**

- Có phương sai của list tần số lấy trên tất cả các bài báo lớn hơn hoặc bằng một giá trị **var_lowerbound**

Giả sử sau khi chọn xong, ta giữ lại được 5 từ (`"kinh tế", "xã hội", "Việt Nam", "Hà Nội", "TPHCM")`, ta có thể gán cho chúng 5 toạ độ, ví dụ `("kinh tế" -> 0, "Hà Nội" -> 1, "TPHCM" -> 2, "Việt Nam" -> 3, "xã hội" -> 4)`. Khi đó túi từ `("Việt Nam":2, "kinh tế":1, "tăng trưởng": 2, "là": 1)` sẽ được biến thành vector `[1, 0, 0, 2, 0]`.

*Hãy viết hàm **`getExplicativeFeatures(global_frequency_file, frequency_file, lowerbound, upperbound, var_lower_bound)`** nhận 5 đối số theo thứ tự là đường dẫn file túi từ theo bài báo, đường dẫn file tần số tổng quát, chặn dưới của tần số cần chọn, chặn trên của tần số cần chọn, chặn dưới của phương sai cần chọn, và trả lại một **dict** của Python gồm các key là các từ "có tính giải thích" được chọn và value tương ứng là số thứ tự của toạ độ.*

Trong ví dụ trên, kết quả trả lại cần là: `{"kinh tế": 0, "Hà Nội": 1, "TPHCM": 2, "Việt Nam": 3, "xã hội": 4}`

Đoạn code dưới đây giúp kiểm tra hàm của bạn.

In [21]:
MIN_AVG = 300
MAX_AVG = 1500
MIN_DEV = 0.8

In [22]:
features = getExplicativeFeatures(GLOBAL_FREQUENCY_FILE, FREQUENCY_FILE, MIN_AVG, MAX_AVG, MIN_DEV)
features

{'Ban': 39,
 'B\xe1\xbb\x99': 6,
 'C': 76,
 'Ch\xc3\xadnh': 108,
 'Qu\xe1\xbb\x91c': 90,
 'TP HCM': 65,
 'Th\xc3\xa0nh': 46,
 'Th\xe1\xbb\xa7': 105,
 'Trung': 75,
 'T\xe1\xbb\x95ng': 22,
 'UBND': 63,
 'Vi\xe1\xbb\x87t Nam': 24,
 'anh': 86,
 'ban': 70,
 'bi\xe1\xbb\x83n': 16,
 'b\xc3\xa0': 110,
 'b\xc3\xa3o': 117,
 'b\xe1\xbb\x99': 122,
 'ch\xc3\xa1y': 114,
 'ch\xc3\xadnh': 26,
 'ch\xe1\xbb\x8b': 101,
 'con': 79,
 'c\xc3\xa1': 126,
 'c\xc3\xa1n b\xe1\xbb\x99': 74,
 'c\xc3\xa2y': 115,
 'c\xc3\xb4ng tr\xc3\xacnh': 9,
 'c\xe1\xba\xa7n': 27,
 'c\xe1\xba\xa7u': 29,
 'c\xe1\xbb\xa9u': 21,
 'doanh': 54,
 'du l\xe1\xbb\x8bch': 58,
 'd\xe1\xbb\xb1 \xc3\xa1n': 77,
 'em': 41,
 'gia \xc4\x91\xc3\xacnh': 125,
 'giao th\xc3\xb4ng': 8,
 'gi\xe1\xba\xa3i': 14,
 'gi\xe1\xbb\x9d': 82,
 'ho\xe1\xba\xa1ch': 17,
 'ho\xe1\xba\xa1t \xc4\x91\xe1\xbb\x99ng': 36,
 'h\xc3\xb3a': 20,
 'h\xe1\xbb\x8dc': 33,
 'h\xe1\xbb\x8dc sinh': 59,
 'h\xe1\xbb\x93': 47,
 'h\xe1\xbb\x99': 48,
 'h\xe1\xbb\x99i': 78,
 'khoa': 61,
 

In [23]:
c1 = pd.Series(features.keys()) 
c2 = pd.Series(features.values()) 
pd.DataFrame([c1, c2]) #Brazil ứng với toạ độ 12, Neymar ứng với toạ độ 9, v.v... Của bạn có thể khác

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,trường,lao động,TP HCM,công trình,viên,trạm,tế,tịch,ương,tướng,...,tôi,khoa,vỉa hè,hoạt động,địa phương,thu,UBND,đất,thi,phát triển
1,100,0,65,9,1,2,3,66,37,4,...,35,61,62,36,96,97,63,64,99,124


### Bài 10. Biến túi từ thành các vector có toạ độ 0/1

Ta cần phân nhóm các bài báo theo độ tương đồng về nội dung. Theo đó, các vector ứng với các bài báo tương đồng (sử dụng chung một lượng từ "quan trọng" giống nhau) cần có khoảng cách nhỏ, còn các vector ứng với các bài báo không tương đồng có khoảng cách lớn. 

Vì điều kiện này, việc số hoá các túi từ thành các vector có toạ độ là tần số các từ không phải là lựa chọn tốt. 

Ví dụ xét 3 túi từ: 

- Túi từ `A {"Việt Nam": 1, "xã hội": 2}`
- Túi từ `B {"Việt Nam": 2, "xã hội": 4}`
- Túi từ `C {"Hà Nội": 1, "TPHCM": 1}`

Nếu biến mỗi túi từ thành vector tần số các từ thì `A, B, C` trở thành `[0, 0, 0, 1, 1]`, `[0, 0, 0, 2, 4]`, `[0, 1, 1, 0, 0]`. Khi đó `AB`$ = \sqrt{10} > $ `AC`$= 2$, trong khi `A` tương đồng với `B` hơn `C`.

Để khắc phục bất tiện này, ta nghĩ đến phương pháp sử dụng phép số hoá 0-1. Nếu một từ "quan trọng" có mặt trong bài báo (túi từ), toạ độ tương ứng của nó bằng 1. Nếu không, toạ độ tương ứng bằng 0.

Với ví dụ trên, `A, B, C` trở thành `[0, 0, 0, 1, 1]`, `[0, 0, 0, 1, 1]`, `[0, 1, 1, 0, 0]`. Khi đó `AB = 0, AC = 2`, một kết quả hợp lí hơn.

*Hãy viết hàm **`articlesToSparseVector(frequency_file, features_dict, coordinates_coding_mode = "0-1")`** nhận đối số **`frequency_file`** là file túi từ, **`features_dict`** là từ điển các từ quan trọng như output của bài 9, **coordinates_coding_mode** là một str cho biết cách số hoá, có giá trị "0-1" trong bài này và sẽ thay đổi ở một số bài sau; và trả lại một numpy array hoặc một sparse matrix hoặc một DataFrame (tuỳ bạn lựa chọn) tương ứng với một ma trận có số hàng bằng số bài báo, mỗi hàng là vector 0-1 đã được chuyển đổi từ túi từ của bài báo đó.*

Đoạn code dưới đây giúp test hàm của bạn (trong đó `X` được chọn là một sparse matrix)

In [24]:
COORDINATES_CODING_MODE = "0-1"
features = getExplicativeFeatures(GLOBAL_FREQUENCY_FILE, FREQUENCY_FILE, MIN_AVG, MAX_AVG, MIN_DEV)
X = articlesToSparseVector(FREQUENCY_FILE, features, COORDINATES_CODING_MODE)
pd.DataFrame(X.toarray()).head()

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,0,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### Bài 11. Hàm phụ - Lấy tiêu đề của các bài báo

*Để tiện cho việc phân tích kết quả sau này, hãy viết hàm **`getTitles(frequency_file)`** nhận đối số **`frequency_file`** là file túi từ, và trả lại list tiêu đề theo đúng thứ tự.*

In [25]:
tt = getTitles(FREQUENCY_FILE)
pd.DataFrame(tt).head(10)

,0
0,Cơ quan khí tượng triều Nguyễn
1,Những chính sách nổi bật có hiệu lực từ tháng 2
2,Đường vành đai Hà Nội ùn tắc ngày mùng 4 Tết
3,Ông Đinh La Thăng: 'CSGT TP HCM phải lấy lại h...
4,Người dân dựng biển cảnh báo 'Công an bắn tốc độ'
5,Nữ sinh bị bạn dùng gạch hành hung đến ngất xỉu
6,"Lễ hội trâu rơm, bò rạ ở đồng bằng sông Hồng"
7,9 câu hỏi về ông vua dời đô từ Hoa Lư về Thăng...
8,Đường hoa Nguyễn Huệ kéo dài thêm một ngày
9,"5 ngày nghỉ Tết, gần 120 người chết vì tai nạn..."


## Phần 3 - KMeans

### Bài 12. Chia nhóm với KMeans và khoảng cách Euclide

*Hãy viết hàm **`train(vectors, nb_clusters, model = "KMeans")`** nhận đối số **`vectors`** là output của bài 10, **`nb_clusters`** là số nhóm cần chia, **`model`** là tên mô hình (trong bài này là **KMeans** by default), và trả lại mô hình dự đoán (predictive model) ứng với thuật toán KMeans với khoảng cách Euclide.*

*Sau đó, hãy viết hàm **`predict(predictive_model, vectors)`** để từ **`predictive_model`** là kết quả của hàm **`train`** trên, trả lại dự đoán của mô hình dành cho **`vectors`** dưới dạng một list gồm các số thứ tự của nhóm tương ứng với từng vector (bài báo).*

Đoạn code dưới đây giúp test hàm của bạn.

In [27]:
COORDINATES_CODING_MODE = "0-1"
NB_CLUSTER = 4
MODEL = "KMeans"
MIN_AVG = 300
MAX_AVG = 1500
MIN_DEV = 0.8

features = getExplicativeFeatures(GLOBAL_FREQUENCY_FILE, FREQUENCY_FILE, MIN_AVG, MAX_AVG, MIN_DEV)
X = articlesToSparseVector(FREQUENCY_FILE, features, COORDINATES_CODING_MODE)
predictive_model = train(X, NB_CLUSTER, model=MODEL)
prediction = predict(predictive_model, X)
prediction

array([3, 1, 3, ..., 2, 3, 3])

In [28]:
titles = getTitles(FREQUENCY_FILE)
c1 = pd.Series(titles)
c2 = pd.Series(prediction)
pd.concat([c1, c2], axis=1).head(10)

,0,1
0,Cơ quan khí tượng triều Nguyễn,3
1,Những chính sách nổi bật có hiệu lực từ tháng 2,1
2,Đường vành đai Hà Nội ùn tắc ngày mùng 4 Tết,3
3,Ông Đinh La Thăng: 'CSGT TP HCM phải lấy lại h...,1
4,Người dân dựng biển cảnh báo 'Công an bắn tốc độ',3
5,Nữ sinh bị bạn dùng gạch hành hung đến ngất xỉu,0
6,"Lễ hội trâu rơm, bò rạ ở đồng bằng sông Hồng",3
7,9 câu hỏi về ông vua dời đô từ Hoa Lư về Thăng...,3
8,Đường hoa Nguyễn Huệ kéo dài thêm một ngày,0
9,"5 ngày nghỉ Tết, gần 120 người chết vì tai nạn...",3


Theo kết quả trên, bài báo 0 và 2 thuộc cùng một nhóm (ta chưa biết vì sao chúng thuộc cùng một nhóm).

### Bài 13. Quan sát kết quả [1] - Nhìn vào từng nhóm

Bảng trên cho phép ta biết bài báo nào được phân vào nhóm nào, nhưng khá "khó nhìn" để phân tích kế quả. Ta tìm cách biểu diễn ở một dạng dễ phân tích hơn. 

*Hãy viết hàm **`getClusters(titles, prediction)`** nhận đối số **`titles`** là list tiêu đề các bài báo theo thứ tự, **`prediction`** là kết quả chia nhóm ở bài 12 dưới dạng một array (như array([0, 2, 0, ..., 4, 2, 3]), sau đó trả lại kết quả là một list gồm $k$ phần tử, trong đó phần tử thứ $i$ là một list tiêu đề các bài báo thuộc nhóm $i$.*

Đoạn code dưới đây giúp test hàm của bạn.

In [29]:
clusters = getClusters(titles, prediction)
pd.DataFrame(clusters[0]).head(10) #Các bài báo thuộc nhóm 0...

,0
0,Nữ sinh bị bạn dùng gạch hành hung đến ngất xỉu
1,Đường hoa Nguyễn Huệ kéo dài thêm một ngày
2,Bia cổ ghi công trạng vị vua anh minh bậc nhất...
3,Vị thần 'trấn Bắc' của kinh thành Thăng Long xưa
4,Cựu chiến binh hiến 2.000 m2 đất mở đường
5,Vụ án 'hóa hổ' giết vua và vị Trạng nguyên đầu...
6,Tên đường ở Sài Gòn xưa được đặt như thế nào
7,Nam sinh người Mông ăn mì tôm dành tiền nghiên...
8,Người Cơ Tu đốt trứng gà chọn đất tốt
9,Công trường khai thác đá cổ xây thành nhà Hồ


In [30]:
pd.DataFrame(clusters[1]).head(10) #Các bài báo thuộc nhóm 1...

,0
0,Những chính sách nổi bật có hiệu lực từ tháng 2
1,Ông Đinh La Thăng: 'CSGT TP HCM phải lấy lại h...
2,Bộ trưởng Khoa học: Cần thay đổi tư duy chiến ...
3,Thứ trưởng Hồ Thị Kim Thoa bị khiển trách
4,Người tuyển dụng 'vụ phó 26 tuổi' bị kiểm tra ...
5,Phần mềm giám định chống trục lợi thẻ bảo hiểm...
6,Thứ trưởng Hồ Thị Kim Thoa bị xem xét kỷ luật ...
7,Thủ tướng: 'Hạn chế xây cao ốc ở trung tâm TP ...
8,Ông Vũ Huy Hoàng bị xóa tư cách nguyên Bộ trưở...
9,Trụ sở nhiều bộ ngành sẽ được xây dựng trên kh...


Như vậy, ta đã gộp tiêu đề các bài báo thuộc cùng một nhóm với nhau để thử xem chúng có liên quan gì. Nhưng cách nhìn này vẫn chưa cho phép phân tích kết quả.

### Bài 14. Quan sát kết quả [2] - Nhìn vào tâm của từng nhóm

Như ta biết, thành phần toạ độ của mỗi vector chỉ có thể là 0 hoặc 1. Do đó tâm của mỗi nhóm là một vector có các thành phần nằm giữa 0 và 1. Thành phần nào càng lớn (càng gần 1) thì các bài báo ứng với nhóm đó càng hay chứa từ quan trọng tương ứng. Nghĩa là các bài báo tương đồng với nhau dựa trên việc dùng chung từ quan trọng đó.

Vậy, tâm của mỗi nhóm cho ta thông tin về nguyên nhân của sự tương đồng trong nhóm.

*Hãy viết hàm **`getClusterCenters(predictive_model, vectors, prediction)`** nhận đối số **`predictive_model`** là mô hình dự đoán đã học (train) từ bài 12, **`vectors`** là output từ bài 10 (ma trận 0-1), **`prediction`** là kết quả của việc chia nhóm (output của bài 12) và trả lại một array hoặc DataFrame gồm $k$ hàng, mỗi hàng là vector tâm của mỗi nhóm.*

Đoạn code dưới đây giúp test hàm của bạn.

In [31]:
centers = getClusterCenters(predictive_model, X, prediction) #Một array hoặc DataFrame
pd.DataFrame(centers)

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,0.110735,0.226543,0.027895,0.295013,0.039730,0.078614,0.110735,0.102282,0.052409,0.103973,...,0.048183,0.036348,0.045647,0.246830,0.293322,0.172443,0.110735,0.224852,0.397295,0.078614
1,0.128008,0.341675,0.042348,0.528393,0.431184,0.312801,0.450433,0.045236,0.148219,0.154957,...,0.052936,0.174206,0.035611,0.178056,0.215592,0.365736,0.078922,0.398460,0.127045,0.053898
2,0.031050,0.102283,0.211872,0.233790,0.156164,0.096804,0.114155,0.090411,0.519635,0.357991,...,0.028311,0.050228,0.223744,0.234703,0.268493,0.265753,0.166210,0.196347,0.058447,0.028311
3,0.027106,0.068785,0.034392,0.078694,0.032935,0.034975,0.039930,0.147187,0.156806,0.042553,...,0.069368,0.015447,0.123579,0.056252,0.107549,0.088895,0.107549,0.038181,0.102303,0.075488


*Hãy viết tiếp hàm **`getExplicatveFeaturesForEachCluster(predictive_model, vectors, prediction, explicative_features)`** nhận các đối số **`predictive_model`** (mô hình học từ hàm **`train`** ở bài 12), **`vectors`** (ma trận  0-1 như output bài 10), **`prediction`** (kết quả chia nhóm như output hàm **`predict`** ở bài 12), **`explicative_features`** (từ điển các từ quan trọng như output bài 9), và trả lại một numpy array hoặc DataFrame gồm $k$ hàng ứng với $k$ cluster, mỗi phần tử là một list các tuple (từ quan trọng, toạ độ tương ứng) xếp theo thứ tự giảm dần của toạ độ.*

Đoạn code sau giúp test hàm của bạn.

In [32]:
features_by_cluster = getExplicatveFeaturesForEachCluster(predictive_model, X, prediction, features)
pd.DataFrame(features_by_cluster)

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,"(học, 0.618765849535)","(trường, 0.485207100592)","(mỗi, 0.42180896027)","(con, 0.409129332206)","(gia đình, 0.39729501268)","(sự, 0.395604395604)","(tôi, 0.392223161454)","(em, 0.377852916314)","(thêm, 0.372781065089)","(rằng, 0.345731191885)",...,"(tướng, 0.039729501268)","(Đảng, 0.039729501268)","(ương, 0.0380388841927)","(thanh tra, 0.0363482671175)","(đô thị, 0.028740490279)","(trạm, 0.0278951817413)","(cháy, 0.0278951817413)","(vỉa hè, 0.0211327134404)","(sân bay, 0.0177514792899)","(tài xế, 0.00676246830093)"
1,"(Chính, 0.611164581328)","(phủ, 0.55822906641)","(hội, 0.557266602502)","(tế, 0.528392685274)","(rằng, 0.528392685274)","(trưởng, 0.526467757459)","(quyền, 0.512030798845)","(cần, 0.48604427334)","(lãnh đạo, 0.484119345525)","(sự, 0.479307025987)",...,"(lũ, 0.0394610202117)","(lộ, 0.0356111645813)","(hồ, 0.0298363811357)","(lớp, 0.0279114533205)","(ngập, 0.0259865255053)","(học sinh, 0.0259865255053)","(cháy, 0.0230991337825)","(vỉa hè, 0.0230991337825)","(chị, 0.0163618864293)","(tài xế, 0.0125120307988)"
2,"(đầu tư, 0.649315068493)","(tuyến, 0.568949771689)","(thông, 0.536073059361)","(dự án, 0.531506849315)","(giao thông, 0.519634703196)","(tỷ, 0.493150684932)","(quận, 0.437442922374)","(đơn vị, 0.3899543379)","(TP HCM, 0.38904109589)","(phí, 0.382648401826)",...,"(bão, 0.0283105022831)","(cá, 0.0283105022831)","(em, 0.0237442922374)","(nghề, 0.0228310502283)","(lớp, 0.02100456621)","(chị, 0.0200913242009)","(lũ, 0.0200913242009)","(trẻ, 0.0191780821918)","(Đảng, 0.013698630137)","(học sinh, 0.013698630137)"
3,"(biển, 0.23754007578)","(khách, 0.192946662781)","(ôtô, 0.176916350918)","(Trung, 0.174293208977)","(máy, 0.170212765957)","(giao thông, 0.156805596036)","(giờ, 0.155639755173)","(mạnh, 0.155348294958)","(anh, 0.152142232585)","(quận, 0.15185077237)",...,"(lao động, 0.0271058000583)","(học sinh, 0.0268143398426)","(thư, 0.0259399591956)","(dự án, 0.0259399591956)","(phí, 0.0238997376858)","(vốn, 0.0204022150976)","(nghiên, 0.0174876129408)","(Đảng, 0.0166132322938)","(thanh tra, 0.0154473914311)","(thi, 0.0154473914311)"


Đến đây ta cảm thấy rõ hơn về cách chia nhóm do KMeans thực hiện:

- Nhóm 0 có vẻ liên quan đến giáo dục và trẻ em (nhìn những từ quan trọng nhất với toạ độ của nó)

- Nhóm 1 có vẻ liên quan đến chính phủ

- Nhóm 2 có vẻ liên quan đến đầu tư dự án

- Nhóm 3 có vẻ liên quan đến giao thông, nhưng để ý rằng toạ độ các từ quan trọng nhỏ hơn hẳn so với 3 nhóm trên. Nhóm này có thể là tổ hợp của một số nhóm khác nhau hoặc bao gồm những từ không phân loại được nhóm.

Thứ tự các nhóm có thể thay đổi trong chương trình của bạn.

## Phần 4 - Thử nghiệm các thao tác khác

### Bài 15 - Thay đổi k

Việc chọn $k=4$ có thể dẫn đến một số nhóm bị gộp vào một nhóm chung (thường là nhóm ở gốc toạ độ 0). Ta thử thay đổi $k$ xem kết quả có tiến triển không.

*Bạn hãy thử thay đổi $k$, tính thế năng tương ứng, vẽ đồ thị thế năng theo $k$ và nhận xét. Không có hàm nào cần viết thêm trong bài này.*

Kết quả có dạng như hình sau:

<img src="https://raw.githubusercontent.com/riduan91/DSC101/master/Lesson5/TD/figure_3.png" width=600></img>

Thực tế, đồ thị trên không cho ta rõ ràng thông tin nên chọn $k$ bằng bao nhiêu. $k=5, 10, 20$ đều có thể là những lựa chọn hợp lí. Ví dụ, với $k = 10.$

In [37]:
WORDLIST_FILE = "VietnameseDictionary/WordList.txt"
FREQUENCY_FILE = "FullData/FrequencyByNewsArticle.txt"
GLOBAL_FREQUENCY_FILE = "FullData/GlobalNewsFrequency.txt"
COORDINATES_CODING_MODE = "0-1"
NB_CLUSTER = 10
MODEL = "KMeans"
MIN_AVG = 300
MAX_AVG = 1500
MIN_DEV = 0.8

features = getExplicativeFeatures(GLOBAL_FREQUENCY_FILE, FREQUENCY_FILE, MIN_AVG, MAX_AVG, MIN_DEV)
X = articlesToSparseVector(FREQUENCY_FILE, features, COORDINATES_CODING_MODE)
titles = getTitles(FREQUENCY_FILE)

predictive_model = train(X, NB_CLUSTER, model=MODEL)
prediction = predict(predictive_model, X)
clusters = getClusters(titles, prediction)

centers = getClusterCenters(predictive_model, X, prediction)
explicativeFeatures = getExplicatveFeaturesForEachCluster(predictive_model, X, prediction, features)

In [41]:
pd.DataFrame(explicativeFeatures)

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,"(khí, 0.867749419954)","(mưa, 0.849187935035)","(Trung, 0.712296983759)","(mạnh, 0.69837587007)","(biển, 0.587006960557)","(bão, 0.517401392111)","(độ, 0.452436194896)","(ương, 0.436194895592)","(lũ, 0.419953596288)","(tăng, 0.408352668213)",...,"(thi, 0.0116009280742)","(tài xế, 0.0092807424594)","(vi, 0.0092807424594)","(xã hội, 0.0046403712297)","(tiền, 0.0046403712297)","(phí, 0.0046403712297)","(phạm, 0.00232018561485)","(thanh tra, 0.00232018561485)","(Đảng, 0.00232018561485)","(vỉa hè, 0.00232018561485)"
1,"(quận, 0.914798206278)","(xử lý, 0.807174887892)","(UBND, 0.7533632287)","(vỉa hè, 0.726457399103)","(vi, 0.717488789238)","(phạm, 0.708520179372)","(đô thị, 0.654708520179)","(phường, 0.645739910314)","(TP HCM, 0.556053811659)","(lãnh đạo, 0.551569506726)",...,"(trạm, 0.0134529147982)","(C, 0.0134529147982)","(học sinh, 0.0134529147982)","(khí, 0.00896860986547)","(sông, 0.00896860986547)","(ngập, 0.00448430493274)","(bão, 0.0)","(lũ, 0.0)","(khoa, 0.0)","(thi, 0.0)"
2,"(học, 0.784114052953)","(trường, 0.77800407332)","(em, 0.682281059063)","(lớp, 0.582484725051)","(học sinh, 0.580448065173)","(con, 0.501018329939)","(gia đình, 0.480651731161)","(tôi, 0.454175152749)","(mỗi, 0.401221995927)","(giờ, 0.362525458248)",...,"(phủ, 0.0244399185336)","(trạm, 0.0224032586558)","(Đảng, 0.0162932790224)","(sân bay, 0.0142566191446)","(đô thị, 0.0142566191446)","(Thủ, 0.0122199592668)","(tướng, 0.010183299389)","(du lịch, 0.010183299389)","(vỉa hè, 0.010183299389)","(tài xế, 0.0020366598778)"
3,"(UBND, 0.5)","(quyền, 0.447791164659)","(tra, 0.410642570281)","(chính, 0.403614457831)","(tịch, 0.386546184739)","(địa phương, 0.36546184739)","(xử lý, 0.347389558233)","(lãnh đạo, 0.34437751004)","(đơn vị, 0.330321285141)","(phòng, 0.295180722892)",...,"(nghiên, 0.035140562249)","(trẻ, 0.0331325301205)","(bão, 0.0321285140562)","(học sinh, 0.031124497992)","(cháy, 0.0281124497992)","(lũ, 0.0240963855422)","(ngập, 0.0220883534137)","(lớp, 0.0190763052209)","(tài xế, 0.0120481927711)","(vỉa hè, 0.0110441767068)"
4,"(ôtô, 0.691158156912)","(giao thông, 0.53798256538)","(tài xế, 0.474470734745)","(máy, 0.415940224159)","(lộ, 0.39103362391)","(khách, 0.328767123288)","(biển, 0.318804483188)","(TP HCM, 0.217932752179)","(mạnh, 0.21295143213)","(quận, 0.211706102117)",...,"(cá, 0.00622665006227)","(lớp, 0.00622665006227)","(phát triển, 0.00622665006227)","(bão, 0.00498132004981)","(lao động, 0.00373599003736)","(ủy, 0.00373599003736)","(thư, 0.00249066002491)","(ương, 0.00124533001245)","(trồng, 0.00124533001245)","(Đảng, 0.00124533001245)"
5,"(học, 0.769857433809)","(cứu, 0.716904276986)","(nghiên, 0.684317718941)","(Việt Nam, 0.668024439919)","(sự, 0.627291242363)","(khoa, 0.584521384929)","(tế, 0.566191446029)","(rằng, 0.533604887984)","(phát triển, 0.521384928717)","(cần, 0.49083503055)",...,"(hồ, 0.0386965376782)","(lộ, 0.030549898167)","(trạm, 0.0285132382892)","(lũ, 0.0264765784114)","(ngập, 0.0244399185336)","(bão, 0.020366598778)","(vỉa hè, 0.0142566191446)","(cháy, 0.0122199592668)","(thanh tra, 0.0081466395112)","(tài xế, 0.0061099796334)"
6,"(con, 0.197175732218)","(tìm, 0.193514644351)","(anh, 0.192991631799)","(Việt Nam, 0.173640167364)","(cứu, 0.167887029289)","(khách, 0.167364016736)","(biển, 0.1589958159)","(quận, 0.155334728033)","(giờ, 0.148535564854)","(gia đình, 0.144874476987)",...,"(thư, 0.0214435146444)","(vỉa hè, 0.0193514644351)","(trạm, 0.0188284518828)","(ủy, 0.0188284518828)","(ôtô, 0.0167364016736)","(tài xế, 0.015690376569)","(thi, 0.015690376569)","(ương, 0.0130753138075)","(Đảng, 0.010460251046)","(thanh tra, 0.00575313807531)"
7,"(Chính, 0.869198312236)","(phủ, 0.852320675105)","(hội, 0.729957805907)","(rằng, 0.677215189873)","(tế, 0.675105485232)","(đầu tư, 0.654008438819)","(cần, 0.651898734177)","(Quốc, 0.594936708861)","(phát triển, 0.592827004219)","(trưởng, 0.569620253165)",...,"(C, 0.0400843881857)","(bão, 0.0379746835443)","(mưa, 0.

Các bài báo tương ứng của các nhóm (lấy khoảng 10 bài đầu tiên)

In [39]:
clusters_as_array = np.array([cluster[:10] for cluster in clusters])
pd.DataFrame(clusters_as_array)

,0,1,2,3,4,5,6,7,8,9
0,"Bắc Bộ giảm nhiệt nhẹ, miền Trung khả năng có mưa","Ngày cuối năm miền Bắc trời rét về đêm, Nam Bộ...",Chọn nước mắm thơm ngon cho bữa cơm ngày Tết,Không khí Hà Nội ô nhiễm nhất thời điểm nào?,Trung Bộ có mưa trên diện rộng,Thời tiết cả nước thuận lợi du xuân dịp Tết,Chủ tịch Hà Nội: ‘Chăm sóc cây xanh như đối vớ...,Miền Bắc rét nhất 4 độ C,Hà Nội sẽ rét 13 độ C,"Đêm nay, miền Bắc đón không khí lạnh mạnh"
1,39 điểm trông giữ xe khu vực Hồ Gươm đêm giao ...,Tân Sơn Nhất 'trong tầm kiểm soát' dù có 120.0...,Tổ công tác đặc biệt túc trực 'giải cứu' kẹt x...,Lãnh đạo cấp vụ cầm đường dây nóng giao thông ...,Ông Đinh La Thăng: 'Quy hoạch huyện lên quận đ...,TP HCM muốn quy hoạch 3 huyện thành quận,"Ôtô đậu trái phép ở Sài Gòn, chuyên gia nói 'c...",Vỉa hè ở Hải Phòng bị tái chiếm khi nhà chức t...,Bộ trưởng Công an: 'Quyết không để tái diễn tì...,Bí thư Hà Nội: 'Lập lại trật tự vỉa hè phải gắ...
2,Nữ sinh bị bạn dùng gạch hành hung đến ngất xỉu,Cựu chiến binh hiến 2.000 m2 đất mở đường,Người Cơ Tu đốt trứng gà chọn đất tốt,Người phụ nữ mở xưởng đóng tàu vươn Hoàng Sa,Trò chơi đánh quay miền sơn cước,"Người đầu tiên mang rau, hoa Đà Lạt về Sài Gòn",Kỷ lục gia sang Nhật biểu diễn cắt tóc bằng kiếm,Thầy giáo 9x đào tạo nhiều học sinh giỏi quốc gia,"'Cột mốc' Trường Sa, đảo Gạc Ma trên đỉnh Trườ...",Chào xuân Đinh Dậu
3,Đường hoa Nguyễn Huệ kéo dài thêm một ngày,Xác pháo vương vãi đường làng ở Hải Phòng,Nhà thờ đá hơn 120 tuổi độc nhất Việt Nam,Làng chài Khe Gà với hải đăng hơn trăm tuổi ở ...,Xác pháo đầy quốc lộ ở Hà Tĩnh,Hà Nội lập đoàn kiểm tra việc chấp hành kỷ cươ...,Cựu binh 20 năm giữ rừng dưới chân núi Hoành Sơn,Chủ tịch Quảng Ngãi viết thư kêu gọi mua hoa g...,Nhiều nhà xe tăng cước 60% dịp Tết,Tiếp viên trả lại gần 100 triệu đồng cho khách...
4,Người dân dựng biển cảnh báo 'Công an bắn tốc độ',"5 ngày nghỉ Tết, gần 120 người chết vì tai nạn...","Xe chở 29 người đi lễ chùa đâm vách núi, 2 ngư...",25 người chết vì tai nạn giao thông trong mùng...,Xe nhích từng mét qua cầu Rạch Miễu ngày mùng ...,3 người tháo chạy khi căn nhà bị ôtô khách tôn...,15 người chết vì tai nạn giao thông trong ngày...,Sài Gòn mưa lớn bất thường ngày Tết,Rửa xe những ngày 'bão giá',"Bắt khách trên cao tốc, 3 tài xế bị phạ..."
5,Ông Đinh La Thăng: 'CSGT TP HCM phải lấy lại h...,Bia cổ ghi công trạng vị vua anh minh bậc nhất...,Vụ án 'hóa hổ' giết vua và vị Trạng nguyên đầu...,Tên đường ở Sài Gòn xưa được đặt như thế nào,Bộ trưởng Khoa học: Cần thay đổi tư duy chiến ...,Nam sinh người Mông ăn mì tôm dành tiền nghiên...,Công trường khai thác đá cổ xây thành nhà Hồ,Từ cậu bé làm ruộng đến giáo sư nổi tiếng ở Mỹ,Chiếc ấn vàng truyền quốc của triều Nguyễn,Chữ 'Thần' bí ẩn trên vách núi ở cửa biển Thần...
6,Cơ quan khí tượng triều Nguyễn,Đường vành đai Hà Nội ùn tắc ngày mùng 4 Tết,"Lễ hội trâu rơm, bò rạ ở đồng bằng sông Hồng",9 câu hỏi về ông vua dời đô từ Hoa Lư về Thăng...,Vị thần 'trấn Bắc' của kinh thành Thăng Long xưa,Đường hoa lớn nhất miền Bắc khoe sắc,Cháy rừng dữ dội ở Hải Phòng,Hà Nội khai trương phố sách Xuân 2017,Ngư dân Quảng Trị thu tiền triệu từ 'lộc biển'...,Học sinh Đồng Nai tri ân 'người đưa đò' ngày T...
7,Bộ Giao thông xây dựng cơ chế làm 1.376 km cao...,Phần mềm giám định chống trục lợi thẻ bảo hiểm...,Thủ tướng: 'Hạn chế xây cao ốc ở trung tâm TP ...,Trụ sở nhiều bộ ngành sẽ được xây dựng trên kh...,Thủ tướng: Tổng cục Tình báo có vị trí đặc biệ...,"Những chính sách, chỉ đạo nổi bật của Chính ph...",Phó thủ tướng chỉ đạo đẩy nhanh dịch vụ công t...,3 huyện ngoại thành TP HCM gần đủ tiêu chí lên...,80 triệu USD đầu tư cho dự án đổi mới giáo dục...,Hiệu trưởng cao đẳng: 'Trường nghề tranh nhau ...
8,5 công trình giao thông kỳ vọng năm 2017 ở TP HCM,3 công trình giao thông hàng chục nghìn tỷ hoà...,Xây dựng trung tâm cơ sở dữ liệu về thu phí kh...,Thủ tướng yêu cầu xử lý nghiêm việc lấn làn xe...,Khởi động dự án cao tốc Tuyên Quang - Phú Thọ,Hà Nội khẳng định 'không có rối loạn' sau điều...,Thông xe tuyến Th

Căn cứ vào kết quả, ta cảm thấy, nếu chia thành 10 nhóm: các bài báo liên quan đến chủ đề sau.

0. Thời tiết, thiên tai
1. Vi phạm, xử lí ở thành phố, quận (cần xem xét các bài báo để hiểu thêm đề tài)
2. Giáo dục, trẻ em
3. Chính quyền địa phương (?)
4. Giao thông
5. Giáo dục, nghiên cứu
6. Nhóm hỗn tạp (toạ độ các cột rất nhỏ)
7. Chính phủ, Quốc hội (chính quyền Trung ương)
8. Đầu tư, dự án
9. Trung ương, Đảng

Cuối cùng, ta tìm size của các nhóm: ta thấy nhóm 6 (hỗn tạp) có size lớn nhất vì nó có thể bao gồm cả các bài báo không phân nhóm được.

In [40]:
[len(cluster) for cluster in clusters]

[431, 223, 491, 996, 803, 491, 1912, 474, 644, 283]

Với k = 20. Ta có kết quả sau:

In [42]:
WORDLIST_FILE = "VietnameseDictionary/WordList.txt"
FREQUENCY_FILE = "FullData/FrequencyByNewsArticle.txt"
GLOBAL_FREQUENCY_FILE = "FullData/GlobalNewsFrequency.txt"
COORDINATES_CODING_MODE = "0-1"
NB_CLUSTER = 20
MODEL = "KMeans"
MIN_AVG = 300
MAX_AVG = 1500
MIN_DEV = 0.8

features = getExplicativeFeatures(GLOBAL_FREQUENCY_FILE, FREQUENCY_FILE, MIN_AVG, MAX_AVG, MIN_DEV)
X = articlesToSparseVector(FREQUENCY_FILE, features, COORDINATES_CODING_MODE)
titles = getTitles(FREQUENCY_FILE)

predictive_model = train(X, NB_CLUSTER, model=MODEL)
prediction = predict(predictive_model, X)
clusters = getClusters(titles, prediction)

centers = getClusterCenters(predictive_model, X, prediction)
explicativeFeatures = getExplicatveFeaturesForEachCluster(predictive_model, X, prediction, features)

pd.DataFrame(explicativeFeatures)

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,"(mưa, 0.859459459459)","(bão, 0.789189189189)","(lũ, 0.735135135135)","(mạnh, 0.713513513514)","(biển, 0.686486486486)","(khí, 0.681081081081)","(sông, 0.589189189189)","(địa phương, 0.589189189189)","(Trung, 0.551351351351)","(Ban, 0.491891891892)",...,"(nghiên, 0.0162162162162)","(vốn, 0.0162162162162)","(tiền, 0.0162162162162)","(vi, 0.0162162162162)","(thi, 0.0162162162162)","(xã hội, 0.00540540540541)","(thanh tra, 0.00540540540541)","(phạm, 0.0)","(Đảng, 0.0)","(vỉa hè, 0.0)"
1,"(tìm, 0.179324894515)","(anh, 0.17194092827)","(con, 0.154008438819)","(cháy, 0.152953586498)","(cứu, 0.137130801688)","(giờ, 0.132911392405)","(đất, 0.130801687764)","(máy, 0.119198312236)","(sông, 0.113924050633)","(phường, 0.10970464135)",...,"(hoạch, 0.0116033755274)","(Thành, 0.0116033755274)","(vỉa hè, 0.0116033755274)","(tướng, 0.0105485232068)","(ương, 0.0084388185654)","(vi, 0.0084388185654)","(Đảng, 0.00632911392405)","(quận, 0.00632911392405)","(thanh tra, 0.00527426160338)","(phạm, 0.0042194092827)"
2,"(Việt Nam, 0.502772643253)","(khách, 0.439926062847)","(sự, 0.343807763401)","(hoạt động, 0.343807763401)","(tế, 0.314232902033)","(Tổng, 0.271719038817)","(thông, 0.266173752311)","(hóa, 0.264325323475)","(đoàn, 0.26247689464)","(hội, 0.260628465804)",...,"(trạm, 0.0184842883549)","(chị, 0.0184842883549)","(trồng, 0.0184842883549)","(tài xế, 0.0184842883549)","(học sinh, 0.0166358595194)","(lũ, 0.0147874306839)","(vi, 0.0129390018484)","(vỉa hè, 0.0129390018484)","(phạm, 0.00739371534196)","(ngập, 0.00739371534196)"
3,"(UBND, 0.837209302326)","(tịch, 0.581395348837)","(quyền, 0.505285412262)","(chính, 0.435517970402)","(tra, 0.380549682875)","(lãnh đạo, 0.359408033827)","(địa phương, 0.357293868922)","(cán bộ, 0.317124735729)","(xử lý, 0.274841437632)","(thông tin, 0.272727272727)",...,"(Việt Nam, 0.0253699788584)","(sân bay, 0.0211416490486)","(tàu, 0.0211416490486)","(C, 0.0211416490486)","(khí, 0.0211416490486)","(lớp, 0.0211416490486)","(bão, 0.0190274841438)","(vỉa hè, 0.0190274841438)","(nghề, 0.0169133192389)","(tài xế, 0.00634249471459)"
4,"(đầu tư, 0.816326530612)","(dự án, 0.707482993197)","(tỷ, 0.641723356009)","(quận, 0.575963718821)","(tuyến, 0.541950113379)","(thông, 0.526077097506)","(giao thông, 0.498866213152)","(hoạch, 0.46485260771)","(công trình, 0.442176870748)","(đô thị, 0.439909297052)",...,"(lũ, 0.0226757369615)","(cá, 0.0226757369615)","(trẻ, 0.0226757369615)","(anh, 0.0226757369615)","(học sinh, 0.0204081632653)","(bão, 0.0181405895692)","(nghề, 0.015873015873)","(Đảng, 0.0113378684807)","(chị, 0.00907029478458)","(tài xế, 0.00453514739229)"
5,"(khí, 0.977777777778)","(mưa, 0.844444444444)","(Trung, 0.844444444444)","(độ, 0.742222222222)","(mạnh, 0.702222222222)","(C, 0.626666666667)","(ương, 0.586666666667)","(biển, 0.528888888889)","(tăng, 0.408888888889)","(mức, 0.382222222222)",...,"(vốn, 0.0)","(thanh tra, 0.0)","(tiền, 0.0)","(Đảng, 0.0)","(tài xế, 0.0)","(sử dụng, 0.0)","(xử lý, 0.0)","(phí, 0.0)","(học sinh, 0.0)","(vỉa hè, 0.0)"
6,"(phí, 0.988165680473)","(thu, 0.976331360947)","(trạm, 0.940828402367)","(đầu tư, 0.881656804734)","(lộ, 0.834319526627)","(thông, 0.816568047337)","(tuyến, 0.786982248521)","(dự án, 0.721893491124)","(giao thông, 0.692307692308)","(mức, 0.591715976331)",...,"(lớp, 0.00591715976331)","(điện, 0.00591715976331)","(lao động, 0.0)","(trồng, 0.0)","(lũ, 0.0)","(nghề, 0.0)","(tàu, 0.0)","(hồ, 0.0)","(Đảng, 0.0)","(ngập, 0.0)"
7,"(phạm, 0.973597359736)","(vi, 0.973597359736)","(xử lý, 0.650165016502)","(tra, 0.561056105611)","(đơn vị, 0.366336633663)","(giao thông, 0.359735973597)","(triệu, 0.316831683168)","(hoạt động, 0.313531353135)","(doanh, 0.293729372937)","(UBND, 0.270627062706)",...,"(C, 0.023102310231)","(học sinh, 0.023102310231)","(thi, 0.023102310231)","(mạnh, 0.019801980198)","(trẻ, 0.019801980198)","(em, 0.013201320132)","(mưa, 0.00990099009901)","(lũ, 0.00660066006601)","(ngập, 0.00660066006601)","(

Ta thấy so với trường hợp $k=10$, có các hiện tượng: 
- 1 cluster bị tách thành nhiều cluster: Ví dụ, cluster thời tiết ở trường hợp $k=10$ bị tách thành các cluster 0, 5.
- Xuất hiện các cluster mới bị tách ra từ cluster hỗn tạp hoặc 1 cluster lớn cũ: ví dụ cluster 18 liên quan đến biển và tàu; cluster 8 liên quan đến nông nghiệp nhưng không xuất hiện trong trường hợp $k=10$.

Bạn có thể chọn $k$ giữa 10 và 20 và có những nhận xét tương tự.

### Bài 16. Thay đổi cách tính khoảng cách

Một hiện tượng có thể phát sinh khi dùng khoảng cách 0-1 là việc không để ý đến tần số của các từ trong bài báo có thể khiến một số từ kém quan trọng trở nên quan trọng. Hiện tượng này chưa được tìm hiểu có xuất hiện trong dữ liệu của ta hay không, nhưng xuất hiện trong dữ liệu về thể thao, bạn có thể tìm hiểu ở TD5 dành cho thể thao nếu quan tâm.

Như vậy, biến mỗi bài báo thành một vector 0-1 có thể là một cách quá đơn giản và không hiệu quả trong một số trường hợp.

Trước đó ta cũng thấy việc số hoá mỗi bài báo thành một vector chứa tần số từ là không hợp lí.

Ta tìm cách dung hoà hai phương pháp này bằng cách xây dựng một hàm `f` biến tần số $a$ mỗi từ thành thành phần toạ độ tương ứng với từ đó, sao cho:

Ứng với 3 bài báo `A`, `B`, `C` có tần số `a`, `b`, `c`.

- Nếu `a = 0 thì f(a) = 0`

- Nếu `a = 0`, `b, c > 0` thì `|f(b) - f(c)| < |f(b) - f(a)|` (Hai bài báo cùng chứa 1 từ có thành phần khoảng cách tương ứng nhỏ hơn 1 bài báo chứa và 1 báo không chứa)

- Nếu `a > b` thì  `f(a) > f(b)` (Tần số càng lớn, toạ độ càng lớn)

- Nếu `a = 1, b` rất lớn, `c = 0` thì `|f(b) - f(a)| \approx |f(c) - f(a)|`. Nghĩa là ta xem như "có 1" là trung bình cộng giữa "không có" và "có rất nhiều".

Theo cách này, ta có thể cho chẳng hạn $f(1) = 2$, đặt một chặn trên tại 4 và xây dựng 1 hàm tăng nhận giá trị trong [2, 4]. Điều kiện thứ 3 cho thấy tần số của 1 từ càng lớn thì độ liên quan đến từ đó càng lớn

Có thể giả sử rằng 1 từ không xuất hiện quá 8 lần trong bài báo, ta có thể chọn hàm $f(a) = 1+ \sqrt{a}$. (Có nhiều cách lựa chọn khác, ví dụ $f(a) = \log_2{(3 + a)}$)

*Hãy sửa chữa hàm **`freqToCoordinates`** đã viết ở bài 10, để khi đối số **`coordinates_coding_mode`** nhận giá trị **"sqrt"** thì output của hàm này sẽ là một ma trận mà mỗi hàng là các vector có thành phần toạ độ được tính theo hàm $f(a) = 1 + \sqrt{a}$.*

Đoạn code dưới đây giúp test hàm của bạn.

In [43]:
WORDLIST_FILE = "VietnameseDictionary/WordList.txt"
FREQUENCY_FILE = "FullData/FrequencyByNewsArticle.txt"
GLOBAL_FREQUENCY_FILE = "FullData/GlobalNewsFrequency.txt"
COORDINATES_CODING_MODE = "sqrt"
NB_CLUSTER = 10
MODEL = "KMeans"
MIN_AVG = 300
MAX_AVG = 1500
MIN_DEV = 0.8

features = getExplicativeFeatures(GLOBAL_FREQUENCY_FILE, FREQUENCY_FILE, MIN_AVG, MAX_AVG, MIN_DEV)
X = articlesToSparseVector(FREQUENCY_FILE, features, COORDINATES_CODING_MODE)
titles = getTitles(FREQUENCY_FILE)

predictive_model = train(X, NB_CLUSTER, model=MODEL)
prediction = predict(predictive_model, X)
clusters = getClusters(titles, prediction)

centers = getClusterCenters(predictive_model, X, prediction)
explicativeFeatures = getExplicatveFeaturesForEachCluster(predictive_model, X, prediction, features)

In [44]:
pd.DataFrame(explicativeFeatures)

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,"(học, 3.34943952534)","(trường, 2.81038093591)","(học sinh, 1.91108589443)","(em, 1.68792132206)","(lớp, 1.46743381884)","(khoa, 1.23090397151)","(thi, 1.18338806119)","(con, 1.07152205969)","(sự, 1.05937612139)","(Đại, 1.03761930152)",...,"(ngập, 0.0658479144876)","(đô thị, 0.0650426446199)","(hồ, 0.0530674655426)","(cháy, 0.0458704216268)","(bão, 0.0401261721057)","(sân bay, 0.0335655275365)","(trạm, 0.0312672745388)","(tàu, 0.0234889959322)","(tài xế, 0.0102647522941)","(vỉa hè, 0.00957530056914)"
1,"(mưa, 2.40173551975)","(khí, 2.31948422887)","(mạnh, 1.9870310817)","(bão, 1.96260943316)","(Trung, 1.7176347139)","(biển, 1.66764457551)","(độ, 1.57998495154)","(C, 1.31913497952)","(lũ, 1.13259255436)","(tăng, 0.946670841233)",...,"(vi, 0.0195599022005)","(phí, 0.0164597819256)","(tiền, 0.0146699266504)","(thi, 0.0138037929366)","(tài xế, 0.00977995110024)","(phạm, 0.00488997555012)","(xã hội, 0.00488997555012)","(thanh tra, 0.00488997555012)","(Đảng, 0.0)","(vỉa hè, 0.0)"
2,"(mỗi, 1.46874745469)","(con, 1.32213002108)","(gia đình, 1.19823549228)","(anh, 1.19323615662)","(cây, 1.18473175888)","(triệu, 1.18397613645)","(tôi, 1.17543883742)","(đất, 1.02212403219)","(trồng, 0.957992165648)","(thêm, 0.932250186237)",...,"(vỉa hè, 0.0792040583156)","(ủy, 0.0791526230436)","(Thủ, 0.0749988699439)","(vi, 0.0706235859566)","(phạm, 0.0686155707237)","(ương, 0.0683280875263)","(Đảng, 0.0574036937594)","(thanh tra, 0.038729421923)","(sân bay, 0.03777725142)","(tài xế, 0.0169158754167)"
3,"(Việt Nam, 0.450902925358)","(tàu, 0.40811640257)","(tìm, 0.402411439634)","(khách, 0.397719915089)","(cứu, 0.386974515994)","(UBND, 0.385224280475)","(con, 0.38075057389)","(biển, 0.374583841918)","(tịch, 0.363343651003)","(Trung, 0.335356217706)",...,"(ương, 0.057247472568)","(ôtô, 0.0559234694521)","(Đảng, 0.0556039731472)","(trồng, 0.0535632590619)","(đô thị, 0.0489124215059)","(vốn, 0.0484934399369)","(trạm, 0.0481817077687)","(thanh tra, 0.04053845239)","(tài xế, 0.0345792062881)","(vỉa hè, 0.0311320528592)"
4,"(Trung, 2.71026641688)","(ương, 2.54339736462)","(ủy, 2.53530657902)","(Ban, 2.38311777777)","(Đảng, 2.28141054526)","(ban, 2.25377040727)","(thư, 2.20120268276)","(cán bộ, 2.19851410123)","(tra, 1.93107125552)","(vi, 1.81361453917)",...,"(cây, 0.0202486860659)","(tuyến, 0.0202486860659)","(bão, 0.0183486238532)","(học sinh, 0.0183486238532)","(tài xế, 0.012532343154)","(trồng, 0.0110743741393)","(tàu, 0.00917431192661)","(ngập, 0.00917431192661)","(vỉa hè, 0.00917431192661)","(hồ, 0.0)"
5,"(phí, 4.06186659841)","(thu, 3.7804769533)","(trạm, 3.57647806596)","(đầu tư, 2.86399687759)","(lộ, 2.40729405998)","(dự án, 2.4007600122)","(thông, 2.25317325816)","(tuyến, 2.1245325728)","(giao thông, 1.75835980561)","(mức, 1.66373029308)",...,"(trẻ, 0.0113636363636)","(cây, 0.0113636363636)","(lớp, 0.0113636363636)","(ngập, 0.0113636363636)","(điện, 0.0113636363636)","(trồng, 0.0)","(lũ, 0.0)","(nghề, 0.0)","(tàu, 0.0)","(hồ, 0.0)"
6,"(xử lý, 1.75078360744)","(UBND, 1.73044495552)","(quận, 1.69609495604)","(tra, 1.42669859212)","(vi, 1.35674227499)","(phạm, 1.31151165748)","(phường, 1.16298996254)","(lãnh đạo, 1.14134548383)","(quyền, 1.1347224667)","(tịch, 1.07211365964)",...,"(học sinh, 0.0669484458631)","(mạnh, 0.0587270610461)","(thi, 0.0555893788503)","(khoa, 0.0527085896514)","(trẻ, 0.0497460267822)","(ngập, 0.0448504962882)","(C, 0.0414065552235)","(lớp, 0.0387222742595)","(bão, 0.0382166030424)","(lũ, 0.0107334525939)"
7,"(Chính, 1.93320639435)","(hội, 1.88902475171)","(phủ, 1.86663725385)","(tế, 1.75801682738)","(phát triển, 1.60211252264)","(trưởng, 1.58717972755)","(rằng, 1.57655284365)","(Việt Nam, 1.54018943193)","(cần, 1.52769281638)","(Bộ, 1.50804575768)",...,"(lũ, 0.0671258062378)","(ngập, 0.0659930403416)","(lộ, 0.0646453422298)","(lớp, 0.0607770066205)","(học sinh, 0.0485156321847)","(vỉa hè, 0.0463456273488)","(hồ, 0.0351807157056)","(cháy, 0.0338640775625)","(tài 

In [46]:
clusters_as_array = np.array([cluster[:10] for cluster in clusters])
pd.DataFrame(clusters_as_array)

,0,1,2,3,4,5,6,7,8,9
0,Nữ sinh bị bạn dùng gạch hành hung đến ngất xỉu,Vụ án 'hóa hổ' giết vua và vị Trạng nguyên đầu...,Tên đường ở Sài Gòn xưa được đặt như thế nào,Nam sinh người Mông ăn mì tôm dành tiền nghiên...,Từ cậu bé làm ruộng đến giáo sư nổi tiếng ở Mỹ,Thầy giáo 9x đào tạo nhiều học sinh giỏi quốc gia,"'Cột mốc' Trường Sa, đảo Gạc Ma trên đỉnh Trườ...",Nam sinh 12 tuổi ẵm nhiều giải thưởng,'Cây đại thụ' của ngành sử Việt Nam Đinh Xuân ...,Giáo viên Thanh Hóa đã nhận lương thưởng Tết
1,"Bắc Bộ giảm nhiệt nhẹ, miền Trung khả năng có mưa","Ngày cuối năm miền Bắc trời rét về đêm, Nam Bộ...",Không khí Hà Nội ô nhiễm nhất thời điểm nào?,Trung Bộ có mưa trên diện rộng,Thời tiết cả nước thuận lợi du xuân dịp Tết,Miền Bắc rét nhất 4 độ C,Hà Nội sẽ rét 13 độ C,"Đêm nay, miền Bắc đón không khí lạnh mạnh",Miền Bắc sắp đón đợt rét mạnh,"Miền Bắc tiếp tục rét, Nam Bộ có mưa to"
2,Cựu chiến binh hiến 2.000 m2 đất mở đường,Ngư dân Quảng Trị thu tiền triệu từ 'lộc biển'...,Người Cơ Tu đốt trứng gà chọn đất tốt,Công trường khai thác đá cổ xây thành nhà Hồ,Người phụ nữ mở xưởng đóng tàu vươn Hoàng Sa,Nhà thờ đá hơn 120 tuổi độc nhất Việt Nam,Trò chơi đánh quay miền sơn cước,"Người đầu tiên mang rau, hoa Đà Lạt về Sài Gòn",Làng chài Khe Gà với hải đăng hơn trăm tuổi ở ...,Thú chơi giống gà nhỏ nhất thế giới ở Sài Gòn
3,Cơ quan khí tượng triều Nguyễn,Đường vành đai Hà Nội ùn tắc ngày mùng 4 Tết,"Lễ hội trâu rơm, bò rạ ở đồng bằng sông Hồng",9 câu hỏi về ông vua dời đô từ Hoa Lư về Thăng...,Vị thần 'trấn Bắc' của kinh thành Thăng Long xưa,Đường hoa lớn nhất miền Bắc khoe sắc,Cháy rừng dữ dội ở Hải Phòng,Hà Nội khai trương phố sách Xuân 2017,Học sinh Đồng Nai tri ân 'người đưa đò' ngày T...,"Sờ đầu rùa, rải tiền lẻ cầu may ở Văn Miếu - Q..."
4,Thứ trưởng Hồ Thị Kim Thoa bị khiển trách,Người tuyển dụng 'vụ phó 26 tuổi' bị kiểm tra ...,Thứ trưởng Hồ Thị Kim Thoa bị xem xét kỷ luật ...,Ông Vũ Huy Hoàng bị xóa tư cách nguyên Bộ trưở...,Trưng bày hơn 200 kỷ vật về Tổng bí thư Trường...,Hai Thứ trưởng Nội vụ bị kỷ luật,Nữ Bí thư Thành đoàn bị kỷ luật vì nhờ người đ...,Ban thường vụ Đảng ủy ngoài nước bị khiển trách,Vụ trưởng Bộ Tài chính làm Phó ban kinh tế Tru...,Bài toán nhân sự ngày Đà Nẵng chia tay Quảng Nam
5,Xây dựng trung tâm cơ sở dữ liệu về thu phí kh...,Bộ Giao thông xây dựng cơ chế làm 1.376 km cao...,"Quảng Trị đề nghị giảm phí, dời trạm BOT",Bộ Giao thông phản hồi việc giảm thời gian thu...,Giám đốc Sở Giao thông Hà Nội thuyết phục các ...,Khởi công mở rộng hầm Hải Vân 2,Bộ trưởng Giao thông đốc thúc thu phí không dừ...,Đề xuất tăng phí cao tốc TP HCM - Long Thành v...,Doanh nghiệp Nhật 'choáng' vì phí cảng biển ở ...,Phát hiện chênh lệch thu phí ở dự án BOT Hà Nộ...
6,Những chính sách nổi bật có hiệu lực từ tháng 2,Đường hoa Nguyễn Huệ kéo dài thêm một ngày,Xác pháo đầy quốc lộ ở Hà Tĩnh,39 điểm trông giữ xe khu vực Hồ Gươm đêm giao ...,Tân Sơn Nhất 'trong tầm kiểm soát' dù có 120.0...,Nhiều nhà xe tăng cước 60% dịp Tết,Phó chủ tịch Hà Nội: Không chủ trương rung chu...,Sở có biên chế 44 cán bộ bổ nhiệm 'phù hợp quy...,Lãnh đạo huyện Bình Chánh: 'Quy hoạch thành qu...,Lãnh đạo cấp vụ cầm đường dây nóng giao thông ...
7,Ông Đinh La Thăng: 'CSGT TP HCM phải lấy lại h...,Bia cổ ghi công trạng vị vua anh minh bậc nhất...,Bộ trưởng Khoa học: Cần thay đổi tư duy chiến ...,Phần mềm giám định chống trục lợi thẻ bảo hiểm...,Thủ tướng: 'Hạn chế xây cao ốc ở trung tâm TP ...,Trụ sở nhiều bộ ngành sẽ được xây dựng trên kh...,Thủ tướng: Tổng cục Tình báo có vị trí đặc biệ...,"Những chính sách, chỉ đạo nổi bật của Chính ph...",Phó thủ tướng chỉ đạo đẩy nhanh dịch vụ công t...,Ông Võ Văn Thưởng: Báo chí sẽ tụt hậu nếu khôn...
8,Người dân dựng biển cảnh báo 'Công an bắn tốc độ',"5 ngày nghỉ Tết, gần 120 người chết vì tai nạn...","Xe chở 29 người đi lễ chùa đâm vách núi, 2 ngư...",25 người chết vì tai nạn giao thông trong mùng...,Xe nhích từng mét qua cầu Rạch Miễu ngày mùng ...,Sài Gòn khác lạ ngày đầu năm,3 người tháo chạy khi căn nhà bị ôtô khách

Việc sử dụng khoảng cách khác 0-1 có lợi ích: ví dụ, thông qua clustering với 10 nhóm, nhóm nào có toạ độ thành phần quan trọng lớn (cỡ >2.5, như nhóm 0, 1, 4, 9) ta xem chúng như 1 cluster có độ thuần nhất cao. Những nhóm có toạ độ cluster trung bình (như 2, 6, 8), chúng có thể là hợp của 1 số cluster con khác. Ví dụ: cluster 2 có khả năng là hợp của 1 cluster "gia đình" và 1 cluster "nông nghiệp". Nếu lọc riêng cluster này và thực hiện lại KMeans, rất có thể ta sẽ tìm ra các cluster con.

Ta kiểm tra lại size của các cluster. Cluster hỗn tạp vẫn có size lớn.

In [47]:
[len(cluster) for cluster in clusters]

[461, 409, 546, 2385, 218, 176, 559, 556, 863, 575]

### Bài 17. KMeans với cosine metric

Trong PC, ta biết rằng thực hiện k-means với cosine metric tương đương với thực hiện k-means với khoảng cách Euclide, nhưng cần preprocess để co dãn mỗi vector thành vector đơn vị.

*Hãy sửa chữa hàm **`train, predict, getClusterCenters, getExplicatveFeaturesForEachCluster`** để khi đối số **`model`** trong hàm **`train`** nhận giá trị **"KMeans_Cosine"** thì thuật toán KMeans với metric cosine được thực hiện. So sánh kết quả với KMeans với khoảng cách Euclide*

Đoạn code dưới đây giúp test hàm của bạn.

In [48]:
WORDLIST_FILE = "VietnameseDictionary/WordList.txt"
FREQUENCY_FILE = "FullData/FrequencyByNewsArticle.txt"
GLOBAL_FREQUENCY_FILE = "FullData/GlobalNewsFrequency.txt"
COORDINATES_CODING_MODE = "sqrt"
NB_CLUSTER = 10
MODEL = "KMeans_Cosine"
MIN_AVG = 300
MAX_AVG = 1500
MIN_DEV = 0.8


features = getExplicativeFeatures(GLOBAL_FREQUENCY_FILE, FREQUENCY_FILE, MIN_AVG, MAX_AVG, MIN_DEV)
X = articlesToSparseVector(FREQUENCY_FILE, features, COORDINATES_CODING_MODE)
titles = getTitles(FREQUENCY_FILE)

predictive_model = train(X, NB_CLUSTER, model=MODEL)
prediction = predict(predictive_model, X)
clusters = getClusters(titles, prediction)

centers = getClusterCenters(predictive_model, X, prediction)
explicativeFeatures = getExplicatveFeaturesForEachCluster(predictive_model, X, prediction, features)
pd.DataFrame(explicativeFeatures)

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,"(tàu, 0.425027879247)","(biển, 0.207557195816)","(cá, 0.131726886203)","(cứu, 0.111612556534)","(tìm, 0.0932773887503)","(Việt Nam, 0.084130748844)","(Trung, 0.0636058310306)","(máy, 0.0594226030898)","(khách, 0.0588677472427)","(đoàn, 0.0574035673474)",...,"(hồ, 0.00254248472715)","(nghiên, 0.00237139804783)","(thư, 0.00209776920078)","(C, 0.00198892070077)","(trạm, 0.00157155416507)","(thanh tra, 0.000610433961411)","(thi, 0.000586303922058)","(trồng, 0.000434316533957)","(Đảng, 0.0)","(vỉa hè, 0.0)"
1,"(UBND, 0.1496609819)","(tra, 0.137928379524)","(xử lý, 0.111550332397)","(vi, 0.102092631474)","(tịch, 0.100830870096)","(phạm, 0.100106747034)","(cán bộ, 0.0975802270117)","(lãnh đạo, 0.0960859969327)","(quận, 0.0875295211256)","(quyền, 0.0856548281804)",...,"(nghề, 0.0056917029072)","(mạnh, 0.0055641196748)","(cháy, 0.00535283638084)","(học sinh, 0.00506767085374)","(C, 0.00486487957528)","(lớp, 0.00475887601816)","(bão, 0.00428788028582)","(ngập, 0.00286080046518)","(lũ, 0.00278189517132)","(tàu, 0.00229084788845)"
2,"(trường, 0.258653913991)","(học, 0.242216062514)","(học sinh, 0.176613015551)","(em, 0.168364999073)","(lớp, 0.138601425604)","(thi, 0.0828093821326)","(con, 0.0750761195066)","(điểm, 0.0748217640712)","(phòng, 0.0720620993659)","(khoa, 0.0706131828204)",...,"(Thủ, 0.00501724809317)","(cá, 0.00408921111131)","(cháy, 0.00337944654084)","(đô thị, 0.00318159843544)","(tài xế, 0.0030580175836)","(tướng, 0.00301814060678)","(trạm, 0.00287521849577)","(tàu, 0.00198572416668)","(sân bay, 0.00118371197835)","(vỉa hè, 0.000368699182003)"
3,"(khí, 0.268511706128)","(mưa, 0.256565487676)","(độ, 0.218542178784)","(mạnh, 0.197605922719)","(Trung, 0.192120649208)","(C, 0.172726003877)","(bão, 0.158610287587)","(biển, 0.158155993286)","(ương, 0.109532465604)","(tăng, 0.0940925664809)",...,"(hội, 0.000848768731633)","(cháy, 0.000796183487618)","(xã hội, 0.000582906652692)","(phạm, 0.000576266449173)","(tiền, 0.000448685643883)","(thanh tra, 0.000377100536253)","(nghề, 0.0)","(Đảng, 0.0)","(phí, 0.0)","(vỉa hè, 0.0)"
4,"(khách, 0.357013549187)","(tuyến, 0.10371589336)","(giao thông, 0.0916776788073)","(hoạt động, 0.072613373482)","(thông, 0.0706060528375)","(TP HCM, 0.0666732073288)","(sân bay, 0.0606328908184)","(điểm, 0.0581519429325)","(ôtô, 0.0579326538642)","(biển, 0.056025531486)",...,"(phủ, 0.00441674751716)","(học sinh, 0.00440019234933)","(lao động, 0.00415783429459)","(cá, 0.00400994045621)","(nguyên, 0.003553838099)","(lũ, 0.0035309599934)","(lớp, 0.002850488355)","(trồng, 0.00240637492655)","(ương, 0.00135022932472)","(Đảng, 0.000860231254374)"
5,"(cháy, 0.477750923266)","(cứu, 0.184855004457)","(quận, 0.125228782474)","(phường, 0.0983469383427)","(giờ, 0.0929170099141)","(phòng, 0.0765657682313)","(ôtô, 0.0665709111664)","(điện, 0.0660884508149)","(máy, 0.0636483125902)","(TP HCM, 0.0591469299284)",...,"(phát triển, 0.00103609189795)","(du lịch, 0.000992907716881)","(tàu, 0.00084739660508)","(phạm, 0.0)","(xã hội, 0.0)","(nghiên, 0.0)","(hội, 0.0)","(thanh tra, 0.0)","(vi, 0.0)","(mức, 0.0)"
6,"(con, 0.131278239915)","(anh, 0.109593552663)","(gia đình, 0.0980813981641)","(tìm, 0.0862758177878)","(đất, 0.0841687265553)","(cây, 0.0780173583826)","(tôi, 0.0733207308039)","(mỗi, 0.0657747574104)","(địa phương, 0.0646097522698)","(sông, 0.0634975091531)",...,"(sân bay, 0.00436049476767)","(tướng, 0.00389921522001)","(cháy, 0.00378857116692)","(phạm, 0.00374739692524)","(Thủ, 0.00365546010664)","(vi, 0.003595245166)","(Đảng, 0.00286183455503)","(ương, 0.00243605200584)","(tài xế, 0.00225353471201)","(thanh tra, 0.00138239176123)"
7,"(đầu tư, 0.19560318513)","(dự án, 0.169985797563)","(tỷ, 0.136952517415)","(tuyến, 0.129064603946)","(thông, 0.123450483234)","(phí, 0.116180356167)","(giao thông, 0.111383629331)","(thu, 0.0924882236736)","(vốn, 0.0876700559417)","(mức, 0.0832514200544)",...,"(cháy, 0.0049408024796)","(lũ, 0.00453640458887)","(lớp, 0.00440

So với KMeans với khoảng cách Euclide, khoảng cách Cosine không cho ta một nhóm hỗn tạp ở toạ độ 0 quá lớn. 

In [49]:
[len(cluster) for cluster in clusters]

[343, 1009, 540, 357, 545, 260, 1208, 800, 584, 1102]

Điều này do khoảng cách cosine là khoảng cách giữa các phương. Nhóm hỗn tạp đã bị chia và tách vào 10 nhóm thành phần. Kết quả về mặt nội dung tuy phần lớn tương tự, nhưng có sự xuất hiện của một số nội dung không tìm thấy khi dùng khoảng cách Euclide. 10 nhóm đó là:

0. Tàu, biển
1. UBND, xử lí vi phạm (toạ độ nhỏ, có thể được chia nhóm tiếp)
2. Giáo dục, trẻ em
3. Thời tiết, thiên tai
4. Giao thông
5. Cháy, cứu hộ (nhóm này có size nhỏ nhất)
6. Gia đình + nông nghiệp (toạ độ nhỏ, có thể được chia nhóm tiếp)
7. Đầu tư, dự án
8. Giao thông (có thể được hợp với nhóm 4)
9. Quốc hội, Chính phủ

Đây chỉ là kết quả về mặt tính toán. Tương tự như K-Means, người dùng có thể từ kết quả này, quyết định tăng $k$, phân nhóm các nhóm có size lớn thành nhỏ hơn (ví dụ chia 6 thành 2 nhóm mới, chia 9 thành 1 số nhóm mới), hợp nhóm các nhóm có size nhỏ và có liên quan về nội dung thành lớn hơn (ví dụ gộp 4, 8 thành 1 nhóm).

### Bài 18. Hierarchical Clustering

*Hãy sửa chữa hàm **`train, predict, getClusterCenters, getExplicatveFeaturesForEachCluster`** để khi đối số model trong hàm train nhận giá trị **"Hierarchical_Euclidean"** thì thuật toán Hierachical với khoảng cách Euclide được thực hiện. So sánh kết quả và thời gian chạy với **KMeans.** *

(Kết quả nhìn chung tương tự với KMeans, do cả 2 phương pháp đều dựa trên giả thiết khoảng cách nhỏ khi tương đồng)

## Bình luận thêm

TD3 và 5 minh hoạ một project thực tế nhỏ về xử lí văn bản (text-mining). Bước preprocessing thường mất rất nhiều thời gian. Việc chọn model hợp lí cũng không hiển nhiên, thường qua nhiều bước thử chọn. Do khuôn khổ TD, ta không thực hiện bước dự đoán với các văn bản mới (có thể không lấy từ vnexpress). Bạn có thể tự thực hiện phần này.

Bài toán clustering nhìn chung "ill-defined" vì không có tiêu chuẩn rõ ràng về mặt toán học để đánh giá phương pháp nào là tốt hay xấu. Dựa trên hiểu biết và kinh nghiệm về lĩnh vực, ta có thể chọn ra một mô hình hợp lí.

Mô hình có thể được cải tiến theo nhiều hướng sau:

- Thay đổi cách giảm số chiều của vector bằng cách chọn từ quan trọng (ví dụ, thay đổi chặn trên, dưới của tần số và phương sai. Trong TD, ta dùng các tham số 300, 1500, 0.8. Chúng có thể được thay đổi)

- Thay đổi metric

- Thay đổi cách số hoá vector

- Dùng mô hình khác

- Tái phân nhóm cluster hỗn tạp 1 hoặc nhiều lần...

- Dùng $k$ lớn, rồi hợp các nhóm nhỏ có nội dung liên quan

- Dùng $k$ nhỏ, rồi với từng nhóm lại chia nhóm lần nữa, nhiều lần...

Bạn có thể tự thực hiện một hướng nếu tò mò.

## Tham khảo

[1] Viện Ngôn ngữ học, GS. Hoàng Phê chủ biên, *Từ điển tiếng Việt*, NXB Hồng Đức (2003)

[2] http://vnexpress.net

[3] http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html

[4] http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html